<a href="https://colab.research.google.com/github/dkalenov/trading-bots-and-strategies/blob/main/supertrend_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backtest

## Connections and imports

### Drive Connection & DB

In [1]:
# import pandas as pd
# from google.colab import drive
# import os

# # Подключение к Google Drive
# drive.mount('/content/drive')

# # Указать путь к папке с файлами на Google Drive
# shared_folder_path = '/content/drive/MyDrive/Sloping bot'

# # Указать пути к файлам параметров и результатов бэктеста
# params_file_path = f'{shared_folder_path}/optimized_params.csv'
# results_file_path = f'{shared_folder_path}/backtest_results.csv'

# # Проверка существования файла с оптимизированными параметрами
# if os.path.exists(params_file_path):
#     optimized_params = pd.read_csv(params_file_path)
# else:
#     params_columns = [
#         'Symbol', 'Interval', 'Start Date', 'End Date', 'Test Return [%]',
#         'window_length', 'min_space', 'sloping_atr_length', 'stop_loss_multiplier',
#         'take_profit_multiplier'
#     ]
#     optimized_params = pd.DataFrame(columns=params_columns)

# # Проверка существования файла с результатами бэктеста
# if os.path.exists(results_file_path):
#     backtest_results = pd.read_csv(results_file_path)
# else:
#     backtest_results_columns = [
#         'Symbol', 'Interval', 'Start Date', 'End Date', 'Total Trades', 'Total PNL', 'Percentage PNL (%)',
#         'Win Rate (%)', 'Profit Factor', 'Risk Ratio',
#         'Num Long Trades', 'Num Short Trades', 'Num Profitable Trades', 'Num Unprofitable Trades',
#         'Avg Profit', 'Avg Loss', 'Max Profit', 'Min Profit', 'Max Loss', 'Min Loss',
#         'Avg Long Duration (days)', 'Avg Long Duration (hours)', 'Avg Long Duration (minutes)', 'Avg Long Duration (seconds)',
#         'Avg Short Duration (days)', 'Avg Short Duration (hours)', 'Avg Short Duration (minutes)', 'Avg Short Duration (seconds)',
#         'window_length', 'min_space', 'sloping_atr_length', 'stop_loss_multiplier', 'take_profit_multiplier'
#     ]
#     backtest_results = pd.DataFrame(columns=backtest_results_columns)




### Imports

In [2]:
!pip install pandas_ta
import pandas_ta as ta
!pip install backtesting
!pip install hyperopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=cb4aef0f38c224ff8fed22debcf57def602aef1b5e6c0c0a82e849a1f754c37b
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=1c12eeadcbf70225569697ab9afca14a5af6fef61ca68cf353c95b95111b9e49
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [3]:
# !python --version

In [4]:
# # УСТАНОВКА TA-LIB В COLAB
# # Устанавливаем предварительно скомпилированные файлы библиотеки TA-Lib
# !pip install pandas==1.5.3
# !wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
# !tar -xzf ta-lib-0.4.0-src.tar.gz
# %cd ta-lib
# !./configure --prefix=/usr
# !make
# !make install
# %cd ..
# !pip install ta-lib

In [5]:
# !pip install pandas==1.5.3
!pip install pandas==2.2.2

In [6]:
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import os
import requests
import zipfile
import csv
import io
import backtesting
from backtesting.lib import crossover

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [7]:
# import pandas as pd

# # Создаем пустой DataFrame с для хранения данных
# columns = [
#     'Symbol', 'Interval', 'Total Trades', 'Total PNL', 'Percentage PNL (%)',
#     'Win Rate (%)', 'Profit Factor', 'Risk Ratio',
#     'Num Long Trades', 'Num Short Trades', 'Num Profitable Trades', 'Num Unprofitable Trades',
#     'Avg Profit', 'Avg Loss', 'Max Profit', 'Min Profit', 'Max Loss', 'Min Loss',
#     'Avg Long Duration (days)', 'Avg Long Duration (hours)', 'Avg Long Duration (minutes)', 'Avg Long Duration (seconds)',
#     'Avg Short Duration (days)', 'Avg Short Duration (hours)', 'Avg Short Duration (minutes)', 'Avg Short Duration (seconds)'
# ]


# sloping_backtest_results = pd.DataFrame(columns=columns)

### Strategy

#### Main

#### Backtesting strategy

In [8]:
class SuperTrend_Strategy(backtesting.Strategy):
    atr_period=7
    multiplier=3.0

    def init(self):
        self.atr = self.I(ta.atr, self.data.High.s, self.data.Low.s, self.data.Close.s, self.atr_period)
        self.up = [0]
        self.down = [0]
        self.trend = [0]

    def next(self):
        hl2 = (self.data.High[-1] + self.data.Low[-1]) / 2
        # вычисляем парамер up и up1 (up прошлой свечи)
        self.up.append(hl2 - self.multiplier * self.atr[-1])
        up1 = self.up[-2] if len(self.up) > 1 else self.up[-1]
        if self.data.Close[-2] > up1:
            self.up[-1] = max(up1, self.up[-1])
        # вычисляем парамер down и down1 (down прошлой свечи)
        self.down.append(hl2 + self.multiplier * self.atr[-1])
        down1 = self.down[-2] if len(self.down) > 1 else self.down[-1]
        if self.data.Close[-2] < down1:
            self.down[-1] = min(down1, self.down[-1])
        # определяем тренд
        if len(self.trend) == 1:
            # изначально у нас тренд бычий (для первой свечи графика)
            self.trend[-1] = 1
        # если тренд медвежий и цена закрытия больше down1
        if self.trend[-1] == -1 and self.data.Close[-1] > down1:
            # то меняем тренд на бычий
            self.trend.append(1)
        # если тренд бычий и цена закрытия меньше up1
        elif self.trend[-1] == 1 and self.data.Close[-1] < up1:
            # то меняем тренд на медвежий
            self.trend.append(-1)
        else:
            # во других случаях тренд сохраняется
            self.trend.append(self.trend[-1])
        # если тренд изменился с медвежьего на бычий, то формируем сингал на покупку
        buy_signal = self.trend[-2] == -1 and self.trend[-1] == 1
        # если тренд изменился с бычего на медвежий, то формируем сингал на продажу
        sell_signal = self.trend[-2] == 1 and self.trend[-1] == -1
        if not self.position and buy_signal:
            self.buy(size=1)
        elif self.position and sell_signal:
            self.position.close()

In [9]:
import numpy as np
from backtesting import Strategy

class SuperTrendStrategy(Strategy):
    atr_period = 10  # Параметры по умолчанию
    multiplier = 3.0

    def init(self):
        # Рассчитываем ATR и сохраняем данные как индикатор
        high = self.data.High
        low = self.data.Low
        close = self.data.Close

        self.atr = self.I(self.calculate_atr, high, low, close, self.atr_period)
        self.upper_band = np.zeros_like(close)
        self.lower_band = np.zeros_like(close)
        self.final_upper_band = np.zeros_like(close)
        self.final_lower_band = np.zeros_like(close)
        self.trend = np.zeros_like(close)

    def next(self):
        high = self.data.High[-1]
        low = self.data.Low[-1]
        close = self.data.Close[-1]
        prev_close = self.data.Close[-2]

        hl2 = (high + low) / 2
        atr = self.atr[-1]

        # Вычисление верхней и нижней границы
        current_upper_band = hl2 + self.multiplier * atr
        current_lower_band = hl2 - self.multiplier * atr

        # Обновление final_upper_band
        if self.upper_band[-2] == 0 or current_upper_band < self.final_upper_band[-2] or close > self.final_upper_band[-2]:
            self.final_upper_band[-1] = current_upper_band
        else:
            self.final_upper_band[-1] = self.final_upper_band[-2]

        # Обновление final_lower_band
        if self.lower_band[-2] == 0 or current_lower_band > self.final_lower_band[-2] or close < self.final_lower_band[-2]:
            self.final_lower_band[-1] = current_lower_band
        else:
            self.final_lower_band[-1] = self.final_lower_band[-2]

        # Определение тренда
        if self.trend[-2] == 0:  # Начальное значение тренда
            self.trend[-1] = 1
        elif self.trend[-2] == 1 and close < self.final_upper_band[-1]:
            self.trend[-1] = -1
        elif self.trend[-2] == -1 and close > self.final_lower_band[-1]:
            self.trend[-1] = 1
        else:
            self.trend[-1] = self.trend[-2]

        # Торговые сигналы
        if self.trend[-2] == -1 and self.trend[-1] == 1:  # Медвежий -> Бычий
            self.buy(size=1)
        elif self.trend[-2] == 1 and self.trend[-1] == -1:  # Бычий -> Медвежий
            self.position.close()

    @staticmethod
    def calculate_atr(high, low, close, period):
        """Функция для расчета ATR"""
        prev_close = np.roll(close, 1)
        tr1 = high - low
        tr2 = np.abs(high - prev_close)
        tr3 = np.abs(low - prev_close)
        tr = np.maximum.reduce([tr1, tr2, tr3])

        atr = np.zeros_like(close)
        atr[period - 1] = np.mean(tr[:period])
        for i in range(period, len(close)):
            atr[i] = ((period - 1) * atr[i - 1] + tr[i]) / period
        return atr


### Functions

#### Get data and gnerate klines

In [10]:
# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    return df

### Backtest

In [11]:
# Загрузка данных
# основные параметры
import pandas as pd

symbol = 'ETHUSDT'
interval = '5m'
start_date = '2023-10'
end_date = '2024-12'

df = download_klines(symbol, interval, start_date, end_date)
print(symbol, interval)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")
df

ETHUSDT 5m
Общее количество строк в загруженных данных: 131904
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-12-31 23:55:00+00:00


,Open,High,Low,Close,Volume
Date,,,,,
2023-10-01 00:00:00+00:00,1670.17,1670.92,1669.49,1670.92,2966.547
2023-10-01 00:05:00+00:00,1670.91,1671.67,1670.59,1671.67,2483.707
2023-10-01 00:10:00+00:00,1671.67,1672.57,1671.67,1672.31,3114.489
2023-10-01 00:15:00+00:00,1672.31,1672.96,1672.19,1672.73,2501.424
2023-10-01 00:20:00+00:00,1672.74,1672.74,1671.65,1672.52,1480.613
...,...,...,...,...,...
2024-12-31 23:35:00+00:00,3336.98,3336.99,3331.00,3332.48,5014.020
2024-12-31 23:40:00+00:00,3332.48,3338.18,3332.47,3338.14,2198.011
2024-12-31 23:45:00+00:00,3338.14,3340.50,3337.34,3339.21,3160.449


In [12]:
class SuperTrend_Strategy(backtesting.Strategy):
    atr_period=20
    multiplier=4.5

    def init(self):
        self.atr = self.I(ta.atr, self.data.High.s, self.data.Low.s, self.data.Close.s, self.atr_period)
        self.up = [0]
        self.down = [0]
        self.trend = [0]

    def next(self):
        hl2 = (self.data.High[-1] + self.data.Low[-1]) / 2
        # вычисляем парамер up и up1 (up прошлой свечи)
        self.up.append(hl2 - self.multiplier * self.atr[-1])
        up1 = self.up[-2] if len(self.up) > 1 else self.up[-1]
        if self.data.Close[-2] > up1:
            self.up[-1] = max(up1, self.up[-1])
        # вычисляем парамер down и down1 (down прошлой свечи)
        self.down.append(hl2 + self.multiplier * self.atr[-1])
        down1 = self.down[-2] if len(self.down) > 1 else self.down[-1]
        if self.data.Close[-2] < down1:
            self.down[-1] = min(down1, self.down[-1])
        # определяем тренд
        if len(self.trend) == 1:
            # изначально у нас тренд бычий (для первой свечи графика)
            self.trend[-1] = 1
        # если тренд медвежий и цена закрытия больше down1
        if self.trend[-1] == -1 and self.data.Close[-1] > down1:
            # то меняем тренд на бычий
            self.trend.append(1)
        # если тренд бычий и цена закрытия меньше up1
        elif self.trend[-1] == 1 and self.data.Close[-1] < up1:
            # то меняем тренд на медвежий
            self.trend.append(-1)
        else:
            # во других случаях тренд сохраняется
            self.trend.append(self.trend[-1])
        # если тренд изменился с медвежьего на бычий, то формируем сингал на покупку
        buy_signal = self.trend[-2] == -1 and self.trend[-1] == 1
        # если тренд изменился с бычего на медвежий, то формируем сингал на продажу
        sell_signal = self.trend[-2] == 1 and self.trend[-1] == -1
        if not self.position and buy_signal:
            self.buy(size=1)
        elif self.position and sell_signal:
            self.position.close()

# запускаем бэктест и выводим результат
test = backtesting.Backtest(data=df, strategy=SuperTrend_Strategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(res)

Start                     2023-10-01 00:00...
End                       2024-12-31 23:55...
Duration                    457 days 23:55:00
Exposure Time [%]                   50.426067
Equity Final [$]                1000411.75171
Equity Peak [$]                 1001876.27062
Return [%]                           0.041175
Buy & Hold Return [%]               99.684605
Return (Ann.) [%]                    0.028592
Volatility (Ann.) [%]                0.126027
Sharpe Ratio                         0.226872
Sortino Ratio                        0.388778
Calmar Ratio                         0.190275
Max. Drawdown [%]                   -0.150266
Avg. Drawdown [%]                   -0.005489
Max. Drawdown Duration      222 days 11:20:00
Avg. Drawdown Duration        3 days 18:37:00
# Trades                                  823
Win Rate [%]                        35.722965
Best Trade [%]                      17.075144
Worst Trade [%]                     -6.066721
Avg. Trade [%]                    

In [13]:
import numpy as np
from backtesting import Strategy


def SuperTrend(high, low, close, period=10, multiplier=3):
    """
    Индикатор SuperTrend для backtesting.

    Параметры:
        high (numpy.ndarray): Массив высоких цен.
        low (numpy.ndarray): Массив низких цен.
        close (numpy.ndarray): Массив закрывающих цен.
        period (int): Период для расчета ATR.
        multiplier (float): Множитель для полос.

    Возвращает:
        numpy.ndarray: Тренд (1 - бычий, -1 - медвежий).
    """
    # Вычисление True Range (TR)
    prev_close = np.roll(close, 1)
    tr = np.maximum.reduce([
        high - low,
        np.abs(high - prev_close),
        np.abs(low - prev_close),
    ])

    # Вычисление ATR (с использованием скользящего среднего)
    atr = np.zeros_like(close)
    atr[:period] = np.nan  # Значения ATR до первого периода - NaN
    atr[period - 1] = np.mean(tr[:period])  # Первое значение ATR как среднее TR
    for i in range(period, len(close)):
        atr[i] = ((atr[i - 1] * (period - 1)) + tr[i]) / period

    # Полосы SuperTrend
    upper_band = (high + low) / 2 + multiplier * atr
    lower_band = (high + low) / 2 - multiplier * atr

    final_upper_band = upper_band.copy()
    final_lower_band = lower_band.copy()

    for i in range(1, len(close)):
        if close[i - 1] <= final_upper_band[i - 1]:
            final_upper_band[i] = min(upper_band[i], final_upper_band[i - 1])
        else:
            final_upper_band[i] = upper_band[i]

        if close[i - 1] >= final_lower_band[i - 1]:
            final_lower_band[i] = max(lower_band[i], final_lower_band[i - 1])
        else:
            final_lower_band[i] = lower_band[i]

    # Определение тренда
    trend = np.zeros_like(close, dtype=int)
    for i in range(1, len(close)):
        if close[i] > final_upper_band[i - 1]:
            trend[i] = 1
        elif close[i] < final_lower_band[i - 1]:
            trend[i] = -1
        else:
            trend[i] = trend[i - 1]  # Сохраняем предыдущий тренд

    return trend



class SuperTrendStrategy(Strategy):
    atr_period = 20
    multiplier = 4.5

    def init(self):
        # Инициализация индикатора SuperTrend
        self.trend = self.I(SuperTrend, self.data.High, self.data.Low, self.data.Close,
                            period=self.atr_period, multiplier=self.multiplier)

    def next(self):
        if len(self.trend) <= 1:
            return  # Пропускаем первые шаги, пока тренд не определен

        trend_prev = self.trend[-2]
        trend_curr = self.trend[-1]

        # Вход в сделку
        if trend_prev == -1 and trend_curr == 1 and not self.position:
            self.buy(size=1)

        # Закрытие сделки
        elif trend_prev == 1 and trend_curr == -1 and self.position:
            self.position.close()



# Пример использования:

test = backtesting.Backtest(data=df, strategy=SuperTrendStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(res)


Start                     2023-10-01 00:00...
End                       2024-12-31 23:55...
Duration                    457 days 23:55:00
Exposure Time [%]                   50.426067
Equity Final [$]                1000411.75171
Equity Peak [$]                 1001876.27062
Return [%]                           0.041175
Buy & Hold Return [%]               99.684605
Return (Ann.) [%]                    0.028592
Volatility (Ann.) [%]                0.126027
Sharpe Ratio                         0.226872
Sortino Ratio                        0.388778
Calmar Ratio                         0.190275
Max. Drawdown [%]                   -0.150266
Avg. Drawdown [%]                   -0.005489
Max. Drawdown Duration      222 days 11:20:00
Avg. Drawdown Duration        3 days 18:37:00
# Trades                                  823
Win Rate [%]                        35.722965
Best Trade [%]                      17.075144
Worst Trade [%]                     -6.066721
Avg. Trade [%]                    

In [ ]:
# Указываем параметры для оптимизации
optimized_results = test.optimize(
    atr_period=range(5, 51, 1),  # Период ATR: от 5 до 50 с шагом 1
    multiplier=[x * 0.1 for x in range(10, 101)],  # Множитель: от 1.0 до 10.0 с шагом 0.1
    maximize='Return [%]',  # Оптимизируем по доходности
    constraint=lambda param: param.multiplier > 1.0  # Дополнительные ограничения (если нужны)
)

# Выводим оптимальные результаты
print("Оптимальные параметры:")
print(symbol, interval, start_date, end_date)
print(optimized_results)
print(optimized_results._strategy)

/usr/local/lib/python3.10/dist-packages/backtesting/backtesting.py:1488: UserWarning: Searching for best of 4140 configurations.
  output = _optimize_grid()


Backtest.optimize:   0%|          | 0/14 [00:00<?, ?it/s]

In [ ]:
optimized_results._strategy

<Strategy SuperTrendStrategy(atr_period=49,multiplier=7.1000000000000005)>

In [ ]:
# atr_period = 14
# multiplier = 4.5



res = test.optimize(maximize='Return [%]', atr_period=range(3, 50), multiplier=range(1, 5))

#### Save data

In [ ]:
import pandas as pd
import os
import shutil

def save_results_to_csv(df, shared_folder_path, file_name):
    csv_file_path = os.path.join(shared_folder_path, file_name)
    temp_file_path = f"{csv_file_path}.temp"

    # Проверка существования файла
    if os.path.exists(csv_file_path):
        # Загружаем текущие данные из файла
        existing_df = pd.read_csv(csv_file_path)

        # Конкатенируем текущие данные с новыми
        combined_df = pd.concat([existing_df, df], ignore_index=True)

        # Удаляем полностью дублирующиеся строки
        combined_df = combined_df.drop_duplicates(keep='first').reset_index(drop=True)
    else:
        print(f"Файл не существует, будет создан новый файл: {csv_file_path}")
        combined_df = df

    # Сохраняем обновленные данные во временный файл
    combined_df.to_csv(temp_file_path, index=False)

    # Резервное копирование старого файла перед заменой
    if os.path.exists(csv_file_path):
        backup_file_path = f"{csv_file_path}.backup"
        shutil.copy(csv_file_path, backup_file_path)
        print(f"Резервная копия создана: {backup_file_path}")

    # Перемещаем временный файл, заменяя старый файл
    os.replace(temp_file_path, csv_file_path)
    print(f"Обновленные данные сохранены в {csv_file_path}")



In [ ]:
import os
import csv
import pandas as pd

# # Функция для сохранения результатов в CSV
# def save_optimized_results_to_csv(optimized_params, path):
#     optimized_params.to_csv(f'{path}/optimized_params.csv', index=False)
#     print("Results saved to CSV.")

# Из-за того, что приходится откатывать pandas для установки ta-lib, может возникать ошибка при сохранении DataFrame в CSV, для решения можно использовать данную функцию

# def save_optimized_results_to_csv(optimized_params, path, file_name):
#     csv_file_path = f'{path}/{file_name}'
#     # Use the built-in CSV library to write to the CSV file
#     with open(csv_file_path, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         # Write header
#         writer.writerow(optimized_params.columns)
#         # Write data
#         for index, row in optimized_params.iterrows():
#             writer.writerow(row)
#     print("Results saved to CSV.")


# import csv

# # Функция для сохранения параметров в CSV с использованием модуля csv
# def save_optimized_results_to_csv(optimized_params, path, file_name):
#     csv_file_path = f'{path}/{file_name}'

#     # Проверяем, существует ли файл
#     file_exists = os.path.isfile(csv_file_path)

#     # Открываем файл в режиме добавления (append), если он существует, иначе в режиме записи (write)
#     with open(csv_file_path, mode='a' if file_exists else 'w', newline='') as file:
#         writer = csv.writer(file)

#         # Записываем заголовки, если файл только что создан
#         if not file_exists:
#             writer.writerow(optimized_params.columns)

#         # Записываем строки из DataFrame
#         for index, row in optimized_params.iterrows():
#             writer.writerow(row)
#     print("Results saved to CSV.")

# Теперь вызов этой функции сохранит данные независимо от версии pandas


In [ ]:
import pandas as pd
import os
import shutil

def update_csv_with_temp_backup(df, csv_file_path):
    temp_file_path = f"{csv_file_path}.temp"

    # Проверка существования файла
    if os.path.exists(csv_file_path):
        # Загружаем текущие данные из файла
        existing_df = pd.read_csv(csv_file_path)

        # Конкатенируем текущие данные с новыми
        combined_df = pd.concat([existing_df, df], ignore_index=True)

        # Удаляем полностью дублирующиеся строки
        combined_df = combined_df.drop_duplicates(keep='first').reset_index(drop=True)
    else:
        print(f"Файл не существует, будет создан новый файл: {csv_file_path}")
        combined_df = df

    # Сохраняем обновленные данные во временный файл
    combined_df.to_csv(temp_file_path, index=False)

    # Резервное копирование старого файла перед заменой
    if os.path.exists(csv_file_path):
        backup_file_path = f"{csv_file_path}.backup"
        shutil.copy(csv_file_path, backup_file_path)
        print(f"Резервная копия создана: {backup_file_path}")

    # Перемещаем временный файл, заменяя старый файл
    os.replace(temp_file_path, csv_file_path)
    print(f"Обновленные данные сохранены в {csv_file_path}")

# Пример использования функции
# Подготовка новых данных для добавления в optimized_params.csv или backtest_results.csv


In [ ]:
# Функция для создания записи параметров
def create_optimized_params_row(best_params, best_loss, symbol, interval, start_date, end_date):
    return {
        'Symbol': symbol,
        'Interval': interval,
        'Start Date': start_date,
        'End Date': end_date,
        'Test Return [%]': best_loss,
        'window_length': best_params['window_length'],
        # 'min_space': best_params['min_space'],
        'sloping_atr_length': best_params['sloping_atr_length'],
        'stop_loss_multiplier': best_params['stop_loss_multiplier'],
        'take_profit_multiplier': best_params['take_profit_multiplier']
    }

#### Optimization

In [ ]:
from hyperopt import fmin, tpe, hp, Trials

def hyperopt_optimizer(data, symbol, interval, start_date, end_date, auto_mode=True, max_evals=5, next_evals=3, shared_folder_path=shared_folder_path):
    global optimized_params
    auto_mode = auto_mode
    attempts = 0
    best_loss = float('inf')
    file_name = 'optimized_params.csv'


    def objective(params):
        window_length = int(params['window_length'])
        min_space = int(params['min_space'])
        sloping_atr_length = int(params['sloping_atr_length'])
        take_profit_multiplier = params['take_profit_multiplier']
        stop_loss_multiplier = params['stop_loss_multiplier']

        bt = backtesting.Backtest(data=data,
                      strategy=SlopingStrategy,
                      cash=1_000_000,
                      commission=0.0005)

        stats = bt.run(
            window_length=window_length,
            # min_space=min_space,
            sloping_atr_length=sloping_atr_length,
            take_profit_multiplier=take_profit_multiplier,
            stop_loss_multiplier=stop_loss_multiplier)

        return -stats['Return [%]']

    space = {
        'window_length': hp.quniform('window_length', 10, 100, 2),
        'min_space': hp.quniform('min_space', 3, 20, 1),
        'sloping_atr_length': hp.quniform('sloping_atr_length', 5, 50, 1),
        'take_profit_multiplier': hp.uniform('take_profit_multiplier', 1.0, 2.5),
        'stop_loss_multiplier': hp.uniform('stop_loss_multiplier', 1.0, 2.5)
    }

    trials = Trials()

    if auto_mode:
      while attempts < 3:
          # Запуск оптимизации
          best_params = fmin(objective,
                      space,
                      algo=tpe.suggest,
                      max_evals=max_evals,
                      trials=trials
                      )

          # Обновление текущей наименьшей убыточности
          current_loss = min(trial['result']['loss'] for trial in trials.trials)
          print("Лучшие параметры:", best_params)


          # Сохраняем данные
          new_row = create_optimized_params_row(best_params, -current_loss, symbol, interval, start_date, end_date)
          # Ensure new_row is not empty before concatenation
          if any(new_row.values()):
              optimized_params = pd.concat([optimized_params, pd.DataFrame([new_row])], ignore_index=True)
          save_results_to_csv(optimized_params, shared_folder_path, file_name)


          # Проверка, нужно ли повторить попытку или завершить оптимизацию
          if current_loss > 50: # если более 50 (50% убыток), то скорее всего мы так и не преблизимся к профиту, лучше заново инициализировать поиск
              attempts += 1
              print(f'Текущий Return [%] {-current_loss}. Сликшом далеко до профита')
              trials = Trials()  # Очищаем trials для новой попытки
              continue

          else:
              # Успешная попытка, продолжаем оптимизацию
              while current_loss < best_loss:
                  best_loss = current_loss

                  best_params = fmin(objective,
                              space,
                              algo=tpe.suggest,
                              max_evals=max_evals + next_evals,
                              trials=trials
                              )

                  max_evals += next_evals
                  current_loss = min(trial['result']['loss'] for trial in trials.trials)
                  print("Лучшие параметры:", best_params)

                  # Сохраняем данные
                  if current_loss < best_loss:
                    new_row = create_optimized_params_row(best_params, -current_loss, symbol, interval, start_date, end_date)
                    # Ensure new_row is not empty before concatenation
                    if any(new_row.values()):
                        optimized_params = pd.concat([optimized_params, pd.DataFrame([new_row])], ignore_index=True)
                    save_results_to_csv(optimized_params, shared_folder_path, file_name)

              # Завершаем цикл, если не достигнуты минимальные потери
              attempts = 3

    # Вручную задаём количество итераций
    else:
        while True:
            best_params = fmin(objective,
                        space,
                        algo=tpe.suggest,
                        max_evals=max_evals,
                        trials=trials)

            current_loss = min(trial['result']['loss'] for trial in trials.trials)
            print("Лучшие параметры:", best_params)

            new_row = create_optimized_params_row(best_params, -current_loss, symbol, interval, start_date, end_date)
            if any(new_row.values()):
                optimized_params = pd.concat([optimized_params, pd.DataFrame([new_row])], ignore_index=True)
            save_results_to_csv(optimized_params, shared_folder_path, file_name)


            user_input = input("Продолжить поиск? Введите количество дополнительных итераций или 'no', 0 для завершения: ").strip()
            if user_input.lower() in ['no', 'n', '0']:
                print("Manual optimization ended.")
                break

            try:
                continuation_rounds = int(user_input)
                if continuation_rounds <= 0:
                    print("Завершаем подбор параметров.")
                    break

                # Продолжаем оптимизацию
                else:
                  best_loss = current_loss
                  best_params = fmin(objective,
                              space,
                              algo=tpe.suggest,
                              max_evals=max_evals + continuation_rounds,
                              trials=trials)
                  max_evals += continuation_rounds
                  current_loss = min(trial['result']['loss'] for trial in trials.trials)
                  print("Лучшие параметры:", best_params)
                  # Сохраняем параметры
                  if current_loss < best_loss:
                    new_row = create_optimized_params_row(best_params, -current_loss, symbol, interval, start_date, end_date)
                    if any(new_row.values()):
                        optimized_params = pd.concat([optimized_params, pd.DataFrame([new_row])], ignore_index=True)
                    save_results_to_csv(optimized_params, shared_folder_path, file_name)


            except ValueError:
                print("Вы неверно ввели количество итераций. Попробуйте сново")

    return best_params

#### Get Backtest Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

initial_capital = 1_000_000

def get_backtest_data(trades, initial_capital=initial_capital):
    # Разделение сделок на лонг и шорт
    long_trades = trades[trades['Size'] > 0].copy()
    short_trades = trades[trades['Size'] < 0].copy()
    num_long_trades = len(long_trades)
    num_short_trades = len(short_trades)

    # Подсчет прибыльных и убыточных сделок
    profitable_trades = trades[trades['PnL'] > 0]
    unprofitable_trades = trades[trades['PnL'] <= 0]
    num_profitable_trades = len(profitable_trades)
    num_unprofitable_trades = len(unprofitable_trades)

    # Средняя прибыль и убыток
    avg_profit = profitable_trades['PnL'].mean() if num_profitable_trades > 0 else 0
    avg_loss = unprofitable_trades['PnL'].mean() if num_unprofitable_trades > 0 else 0

    # Средняя продолжительность сделок в лонг и шорт
    avg_long_duration = (long_trades['ExitTime'] - long_trades['EntryTime']).mean() if num_long_trades > 0 else pd.Timedelta(0)
    avg_short_duration = (short_trades['ExitTime'] - short_trades['EntryTime']).mean() if num_short_trades > 0 else pd.Timedelta(0)

    # Общая прибыль/убыток и процентное изменение от начального капитала
    total_profit = trades['PnL'].sum()
    percentage_profit_loss = (total_profit / initial_capital) * 100

    # Соотношение риска и интерпретация
    risk_ratio = abs(avg_loss / avg_profit) if avg_profit != 0 else float('inf')

    # Метрики продолжительности сделок
    avg_long_days, avg_long_hours, avg_long_minutes, avg_long_seconds = avg_long_duration.components[:4]
    avg_short_days, avg_short_hours, avg_short_minutes, avg_short_seconds = avg_short_duration.components[:4]

    # Дополнительные метрики
    max_profit = profitable_trades['PnL'].max() if num_profitable_trades > 0 else 0
    min_profit = profitable_trades['PnL'].min() if num_profitable_trades > 0 else 0
    max_loss = unprofitable_trades['PnL'].min() if num_unprofitable_trades > 0 else 0
    min_loss = unprofitable_trades['PnL'].max() if num_unprofitable_trades > 0 else 0
    win_rate = (num_profitable_trades / len(trades)) * 100 if len(trades) > 0 else 0
    profit_factor = profitable_trades['PnL'].sum() / abs(unprofitable_trades['PnL'].sum()) if num_unprofitable_trades > 0 else float('inf')

    # Вывод всех результатов
    print("Результаты backtest:")
    print(f"Total Trades: {len(trades)}")
    print(f"Total PNL: {total_profit}")
    print(f"Percentage PNL (%): {percentage_profit_loss:.2f}")
    print(f"Win Rate (%): {win_rate:.2f}")
    print(f"Profit Factor: {profit_factor:.2f}")
    print(f"Risk Ratio: {risk_ratio:.2f}")
    print(f"Num Long Trades: {num_long_trades}")
    print(f"Num Short Trades: {num_short_trades}")
    print(f"Num Profitable Trades: {num_profitable_trades}")
    print(f"Num Unprofitable Trades: {num_unprofitable_trades}")
    print(f"Avg Profit: {avg_profit:.2f}")
    print(f"Avg Loss: {avg_loss:.2f}")
    print(f"Max Profit: {max_profit}")
    print(f"Min Profit: {min_profit}")
    print(f"Max Loss: {max_loss}")
    print(f"Min Loss: {min_loss}")
    print(f"Avg Long Duration {avg_long_days} days, {avg_long_hours} hours, {avg_long_minutes} minutes, {avg_long_seconds} seconds")
    print(f"Avg Short Duration {avg_short_days} days, {avg_short_hours} hours, {avg_short_minutes} minutes, {avg_short_seconds} seconds")

    # Возвращение результатов в виде словаря
    return {
        'Total Trades': len(trades),
        'Total PNL': total_profit,
        'Percentage PNL (%)': percentage_profit_loss,
        'Win Rate (%)': win_rate,
        'Profit Factor': profit_factor,
        'Risk Ratio': risk_ratio,
        'Num Long Trades': num_long_trades,
        'Num Short Trades': num_short_trades,
        'Num Profitable Trades': num_profitable_trades,
        'Num Unprofitable Trades': num_unprofitable_trades,
        'Avg Profit': avg_profit,
        'Avg Loss': avg_loss,
        'Max Profit': max_profit,
        'Min Profit': min_profit,
        'Max Loss': max_loss,
        'Min Loss': min_loss,
        'Avg Long Duration (days)': avg_long_days,
        'Avg Long Duration (hours)': avg_long_hours,
        'Avg Long Duration (minutes)': avg_long_minutes,
        'Avg Long Duration (seconds)': avg_long_seconds,
        'Avg Short Duration (days)': avg_short_days,
        'Avg Short Duration (hours)': avg_short_hours,
        'Avg Short Duration (minutes)': avg_short_minutes,
        'Avg Short Duration (seconds)': avg_short_seconds
    }


# Пример вызова функции
# res должен быть объектом с данными о сделках
# Пример:
# result_table = analyze_trades(res, initial_capital=1_000_000, symbol='BTCUSDT', interval='1h')
# print(result_table)


#### Plot Deposit and Price Changes

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio


initial_capital = 1_000_000

def plot_deposit_and_price_change(trades, df, symbol, interval, basis, initial_capital=initial_capital):
    # Изменения депозита
    deposit_changes = [initial_capital]
    for trade in trades.itertuples():
        deposit_changes.append(deposit_changes[-1] + trade.PnL)

    # Находим начало и конец датасета
    start_date = min(df.index.min(), trades['EntryTime'].min())
    end_date = max(df.index.max(), trades['ExitTime'].max())


    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=f"Изменение депозита, {symbol}")

    # Изменение депозита
    fig.add_trace(go.Scatter(
        x=trades['ExitTime'],
        y=deposit_changes[1:],  # Skip initial capital for plotting
        mode='lines+markers',
        name='Изменение депозита',
        line=dict(color='blue'),
        hovertemplate='%{x}: Депозит: %{y:.2f}<br>Профит/Убыток: %{text}',
        text=[f'Профит: {trade.PnL:.2f}' if trade.PnL > 0 else f'Убыток: {trade.PnL:.2f}' for trade in trades.itertuples()]
    ), row=1, col=1)

    # Цена BTC
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['Close'],
        mode='lines',
        name='Цена BTC',
        line=dict(color='orange'),
        hovertemplate='Дата: %{x}<br>Цена: %{y:.2f}'
    ), row=2, col=1)

    # добавляем маркеры для сделок
    for trade in trades.itertuples():
        # Макрерком для изменения депозита является выход из сделки (ExitTime)
        marker_color = 'green' if trade.PnL > 0 else 'red'
        fig.add_trace(go.Scatter(
            x=[trade.ExitTime],
            y=[deposit_changes[trade.Index + 1]],
            mode='markers',
            marker=dict(color=marker_color, size=8, symbol='circle'),
            name='Профит' if trade.PnL > 0 else 'Убыток',
            hovertemplate=f'Сделка №{trade.Index + 1}: Профит: {trade.PnL:.2f}'
        ), row=1, col=1)

        # Стрелка для графика о движении цены, задаем в точке входа (EntryTime)
        arrow_symbol = '↑' if trade.Size > 0 else '↓'
        arrow_color = 'green' if trade.Size > 0 else 'red'
        fig.add_annotation(
            x=trade.EntryTime,
            y=df.loc[trade.EntryTime, 'Close'],
            text=arrow_symbol,
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-30,
            font=dict(size=20, color=arrow_color),
            hovertext=f'Сделка №{trade.Index + 1} ({"Лонг" if trade.Size > 0 else "Шорт"})',
            hoverlabel=dict(bgcolor='white', bordercolor='black'),
            xref="x",
            yref="y2"
        )

    fig.update_layout(
        title=f'Изменение депозита и цены {symbol} ({interval})',
        xaxis=dict(title='Дата', range=[start_date, end_date]),
        xaxis2=dict(title='Дата', range=[start_date, end_date]),
        yaxis=dict(title='Депозит ($)'),
        yaxis2=dict(title=f'Цена {symbol} ($)'),
        legend=dict(x=0, y=1),
        height=1200,
        width=5000
    )

    fig.update_xaxes(tickangle=-45)
    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.show()

    # Сохраняем гарфики в HTML file
    filename = f"plot_{symbol}_{interval}_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}_basis.html"
    pio.write_html(fig, file=filename)
    print(f"График сохранен как {filename}")




# Пример вызова функции
# plot_deposit_and_price_changes(trades, df, symbol, interval)


#### Backtest Pipeline

In [ ]:
# pipeline для финального бэктеста
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import shutil

# Фильтр для отбора моделей с положительным PnL на заданном интервале
def filter_positive_pnl(df, symbol, interval):
    df.columns = [col.lower().replace(" ", "_") for col in df.columns]
    try:
        filtered_df = df[(df['symbol'] == symbol) & (df['interval'] == interval) & (df['total_pnl'] > 0)]
    except KeyError:
        filtered_df = df[(df['symbol'] == symbol) & (df['interval'] == interval) & (df['test_return_[%]'] > 0)]
    # Reset index and return the DataFrame
    return filtered_df.reset_index(drop=True)

# Генерация точных месячных интервалов
def generate_monthly_intervals(start_date, end_date, interval_count=12):
    intervals = []
    current_date = datetime.strptime(start_date, "%Y-%m")
    end_date = datetime.strptime(end_date, "%Y-%m")

    while current_date < end_date and len(intervals) < interval_count:
        next_date = current_date + relativedelta(months=1)
        intervals.append((current_date.strftime("%Y-%m"), next_date.strftime("%Y-%m")))
        current_date = next_date

    return intervals

# Основная функция бэктестинга с фильтрацией при загрузке
def pipeline(df_backtest_results, symbol, interval, start_date, end_date, strategy, initial_capital=1_000_000, interval_count=12):
    # Фильтрация моделей по профитности
    filtered_models = filter_positive_pnl(df_backtest_results, symbol=symbol, interval=interval)
    if filtered_models.empty:
        print("Нет профитных моделей для данного символа и интервала.")
        return pd.DataFrame()

    intervals = generate_monthly_intervals(start_date, end_date, interval_count)
    pnl_columns = [f"{start} - {end}" for start, end in intervals]

    results_df = pd.DataFrame(columns=[
        'model_name', 'symbol', 'interval', 'window_length', 'atr_length',
        'min_space', 'stop_loss_multiplier', 'take_profit_multiplier'
    ] + pnl_columns + ['total_pnl%', 'profit_month_count', 'loss_month_count'])

    # Загрузка свечей для каждого интервала
    monthly_data = {start: download_klines(symbol, interval, start, end) for start, end in intervals}

    # Бэктест для каждой отфильтрованной модели
    for model_index, row in filtered_models.iterrows():
        total_pnl = 0
        profit_month_count = 0
        loss_month_count = 0

        model_name = f"Model_{model_index+1}_{row['window_length']}.{row['sloping_atr_length']}."\
                     f"{int(row['min_space'] if not pd.isna(row['min_space']) else 5)}.{round(row['stop_loss_multiplier'], 2)}."\
                     f"{round(row['take_profit_multiplier'], 2)}"

        # Заполнение информации о модели
        results_df.loc[model_index, ['model_name', 'symbol', 'interval', 'window_length',
                                     'atr_length', 'min_space', 'stop_loss_multiplier',
                                     'take_profit_multiplier']] = [
            model_name, symbol, interval, row['window_length'], row['sloping_atr_length'],
            int(row['min_space'] if not pd.isna(row['min_space']) else 5), row['stop_loss_multiplier'], row['take_profit_multiplier']
        ]

        # Бэктест по каждому месяцу для текущей модели
        for (start, end), pnl_col in zip(intervals, pnl_columns):
            data = monthly_data[start]

            # Настройка параметров стратегии
            strategy.length = row['window_length']
            strategy.atr_length = row['sloping_atr_length']
            strategy.min_space = int(row['min_space'] if not pd.isna(row['min_space']) else 5)
            strategy.stop_loss_multiplier = row['stop_loss_multiplier']
            strategy.take_profit_multiplier = row['take_profit_multiplier']

            # Запуск бэктеста
            test = backtesting.Backtest(data=data, strategy=strategy, cash=initial_capital, commission=0.0005)
            res = test.run()
            pnl_month = round((res._trades['PnL'].sum() / initial_capital) * 100, 2)

            results_df.at[model_index, pnl_col] = pnl_month
            total_pnl += pnl_month

            # Подсчет профитных и убыточных месяцев
            if pnl_month > 0:
                profit_month_count += 1
            else:
                loss_month_count += 1

            # Промежуточный вывод результатов по текущей модели и интервалу
            print(f"Модель {model_name}, Интервал {start} - {end}: PnL={pnl_month}")

        # Итоговые результаты для модели
        results_df.loc[model_index, 'total_pnl%'] = total_pnl
        results_df.loc[model_index, 'profit_month_count'] = profit_month_count
        results_df.loc[model_index, 'loss_month_count'] = loss_month_count

        # Промежуточный вывод итогов по модели
        print(f"Обработана модель {model_name}: Total PnL%={total_pnl}, Profitable Months={profit_month_count}, Loss Months={loss_month_count}")

    return results_df

#### Hyperopt_by_month

In [ ]:
import pandas as pd
from datetime import datetime
from hyperopt import fmin, tpe, hp, Trials
import backtesting


def generate_monthly_intervals(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m")
    end = datetime.strptime(end_date, "%Y-%m")
    intervals = []
    while start <= end:
        next_month = datetime(start.year + (start.month // 12), (start.month % 12) + 1, 1)
        intervals.append((start.strftime("%Y-%m"), next_month.strftime("%Y-%m")))
        start = next_month
    return intervals

# Полный бэктест
def full_period_backtest(strategy, data, initial_capital=1_000_000):
    data.index = pd.to_datetime(data.index).tz_localize(None)
    test = backtesting.Backtest(data, strategy, cash=initial_capital, commission=0.0005)
    res = test.run()
    trades = res._trades.copy()
    return trades, res

# Унифицированная функция для анализа результатов по месяцам
def analyze_monthly_results(trades, intervals, initial_capital):
    trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
    trades.set_index('Close Time', inplace=True)
    trades['Trade Duration'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds()

    monthly_results = {}
    total_pnl = 0
    profit_months = 0
    loss_months = 0
    total_trade_duration = 0
    total_trades = len(trades)

    for start, end in intervals:
        start_date = datetime.strptime(start, "%Y-%m")
        end_date = datetime.strptime(end, "%Y-%m")

        # Фильтруем сделки для текущего месяца
        monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

        if monthly_trades.empty:
            monthly_results[start] = (0.0, 0)
            continue

        # Вычисляем PnL за месяц
        monthly_pnl = monthly_trades['PnL'].sum() / initial_capital * 100

        # Среднее время сделки
        avg_trade_duration = monthly_trades['Trade Duration'].mean() / 3600

        monthly_results[start] = (round(monthly_pnl, 2), round(avg_trade_duration, 2))

        total_pnl += monthly_pnl
        total_trade_duration += monthly_trades['Trade Duration'].sum()
        if monthly_pnl > 0:
            profit_months += 1
        else:
            loss_months += 1

    avg_trade_duration_total = (total_trade_duration / total_trades) / 3600 if total_trades > 0 else 0

    # Вычисляем процентное соотношение профитных месяцев
    total_months = profit_months + loss_months
    profit_month_ratio = (profit_months / total_months * 100) if total_months > 0 else 0

    return total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration_total, profit_month_ratio

def generate_search_space(params_to_optimize):
    """Генерация пространства поиска для Hyperopt."""
    space = {}
    for param, settings in params_to_optimize.items():
        if settings['type'] == 'quniform':
            space[param] = hp.quniform(param, *settings['range'])
        elif settings['type'] == 'uniform':
            space[param] = hp.uniform(param, *settings['range'])
    return space

def optimize_strategy(data, symbol, interval, start_date, end_date, shared_folder_path, max_evals, params_to_optimize, fixed_params):
    """Функция для оптимизации стратегии с заданными параметрами."""
    intervals = generate_monthly_intervals(start_date, end_date)
    trials = Trials()
    space = generate_search_space(params_to_optimize)

    results = []

    print(f"Общее количество строк в загруженных данных: {len(data)}")
    print(f"Диапазон дат: {data.index.min()} - {data.index.max()}")
    print(f"Оптимизация для {symbol} ({interval}) с {start_date} по {end_date} началась...\n")

    def objective(params):
        # Установка параметров стратегии
        for key, value in fixed_params.items():
            setattr(SlopingStrategy, key, value)

        for key, value in params.items():
            setattr(SlopingStrategy, key, int(value) if 'length' in key else value)

        try:
            trades, res = full_period_backtest(
                strategy=SlopingStrategy,
                data=data,
                initial_capital=1_000_000
            )

            total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration, profit_month_ratio = analyze_monthly_results(
                trades=trades,
                intervals=intervals,
                initial_capital=1_000_000
            )

            print(f"Параметры: {params}, {fixed_params}")
            print(f"Общий профит: {total_pnl:.2f}%, Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}, Процент профитных месяцев: {profit_month_ratio:.2f}%")
            print(f"Результаты по месяцам: {[(month, pnl) for month, (pnl, _) in monthly_results.items()]}\n")

            row = {
                'symbol': symbol,
                'interval': interval,
                **{key: fixed_params.get(key, None) for key in fixed_params},
                **{key: (int(value) if 'length' in key else value) for key, value in params.items()},
                'total_pnl%': total_pnl,
                'avg_trade_duration_hours': avg_trade_duration,
                'profit_month_count': profit_months,
                'loss_month_count': loss_months,
                'profit_month_ratio%': profit_month_ratio
            }

            for month, (pnl, duration) in monthly_results.items():
                row[month] = pnl

            results.append(row)
            return {'loss': -total_pnl, 'status': 'ok', 'params': params}

        except Exception as e:
            print(f"Ошибка в objective: {e}")
            return {'loss': float('inf'), 'status': 'fail', 'params': params}

    try:
        fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    except Exception as e:
        print(f"Ошибка при выполнении fmin: {e}")
        return None

    results_df = pd.DataFrame(results)
    save_results_to_csv(results_df, shared_folder_path, "hyperopt_optimization_results.csv")

    if trials.best_trial:
        best_trial = trials.best_trial['result']
        best_params = best_trial['params']
        best_loss = best_trial['loss']
        return best_params, -best_loss
    else:
        print("Не удалось найти лучшие параметры.")
        return None

In [ ]:
# import pandas as pd
# from datetime import datetime
# from hyperopt import fmin, tpe, hp, Trials
# import backtesting

# # Генерация интервалов
# def generate_monthly_intervals(start_date, end_date):
#     start = datetime.strptime(start_date, "%Y-%m")
#     end = datetime.strptime(end_date, "%Y-%m")
#     intervals = []
#     while start <= end:
#         next_month = datetime(start.year + (start.month // 12), (start.month % 12) + 1, 1)
#         intervals.append((start.strftime("%Y-%m"), next_month.strftime("%Y-%m")))
#         start = next_month
#     return intervals

# # Анализ помесячно
# def analyze_monthly_results(trades, intervals, initial_capital):
#     trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
#     trades.set_index('Close Time', inplace=True)
#     trades['Trade Duration'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds()

#     monthly_results = {}
#     total_pnl = 0
#     profit_months = 0
#     loss_months = 0
#     total_trade_duration = 0
#     total_trades = len(trades)

#     for start, end in intervals:
#         start_date = datetime.strptime(start, "%Y-%m")
#         end_date = datetime.strptime(end, "%Y-%m")

#         # Фильтруем сделки для текущего месяца
#         monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

#         if monthly_trades.empty:
#             monthly_results[start] = (0.0, 0)
#             continue

#         # Вычисляем PnL за месяц
#         monthly_pnl = monthly_trades['PnL'].sum() / initial_capital * 100

#         # Среднее время сделки
#         avg_trade_duration = monthly_trades['Trade Duration'].mean() / 3600

#         monthly_results[start] = (round(monthly_pnl, 2), round(avg_trade_duration, 2))

#         total_pnl += monthly_pnl
#         total_trade_duration += monthly_trades['Trade Duration'].sum()
#         if monthly_pnl > 0:
#             profit_months += 1
#         else:
#             loss_months += 1

#     avg_trade_duration_total = (total_trade_duration / total_trades) / 3600 if total_trades > 0 else 0
#     return total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration_total

# # Полный бэктест
# def full_period_backtest(strategy, data, initial_capital=1_000_000):
#     data.index = pd.to_datetime(data.index).tz_localize(None)
#     test = backtesting.Backtest(data, strategy, cash=initial_capital, commission=0.0005)
#     res = test.run()
#     trades = res._trades.copy()
#     return trades, res


# # Оптимизация стратегии
# def optimize_strategy(data, symbol, interval, start_date, end_date, shared_folder_path, max_evals=10):
#     intervals = generate_monthly_intervals(start_date, end_date)
#     trials = Trials()

#     space = {
#         'window_length': hp.quniform('window_length', 10, 100, 2),
#         'min_space': hp.quniform('min_space', 3, 20, 1),
#         'sloping_atr_length': hp.quniform('sloping_atr_length', 7, 30, 1),
#         'take_profit_multiplier': hp.uniform('take_profit_multiplier', 1.0, 3.1),
#         'stop_loss_multiplier': hp.uniform('stop_loss_multiplier', 0.5, 1.5)
#     }

#     results = []

#     print(f"Общее количество строк в загруженных данных: {len(data)}")
#     print(f"Диапазон дат: {data.index.min()} - {data.index.max()}")
#     print(f"Оптимизация для {symbol} ({interval}) с {start_date} по {end_date} началась...\n")

#     def objective(params):
#         SlopingStrategy.window_length = int(params['window_length'])
#         SlopingStrategy.min_space = int(params['min_space'])
#         SlopingStrategy.sloping_atr_length = int(params['sloping_atr_length'])
#         SlopingStrategy.take_profit_multiplier = params['take_profit_multiplier']
#         SlopingStrategy.stop_loss_multiplier = params['stop_loss_multiplier']

#         try:
#             trades, res = full_period_backtest(
#                 strategy=SlopingStrategy,
#                 data=data,
#                 initial_capital=1_000_000
#             )

#             total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration = analyze_monthly_results(
#                 trades=trades,
#                 intervals=intervals,
#                 initial_capital=1_000_000
#             )

#             print(f"Параметры: {params}")
#             print(f"Общий профит: {total_pnl:.2f}%, Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}")
#             print(f"Результаты по месяцам: {[(month, pnl) for month, (pnl, _) in monthly_results.items()]}\n")

#             # Формируем результат с правильным порядком колонок
#             row = {
#                 'symbol': symbol,
#                 'interval': interval,
#                 'window_length': int(params['window_length']),
#                 'min_space': int(params['min_space']),
#                 'sloping_atr_length': int(params['sloping_atr_length']),
#                 'take_profit_multiplier': params['take_profit_multiplier'],
#                 'stop_loss_multiplier': params['stop_loss_multiplier'],
#                 'total_pnl%': total_pnl,
#                 'avg_trade_duration_hours': avg_trade_duration,
#                 'profit_month_count': profit_months,
#                 'loss_month_count': loss_months
#             }

#             # Добавляем результаты по месяцам
#             for month, (pnl, duration) in monthly_results.items():
#                 row[month] = pnl

#             results.append(row)
#             return {'loss': -total_pnl, 'status': 'ok', 'params': params}

#         except Exception as e:
#             print(f"Ошибка в objective: {e}")
#             return {'loss': float('inf'), 'status': 'fail', 'params': params}
#     try:
#         fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
#     except Exception as e:
#         print(f"Ошибка при выполнении fmin: {e}")
#         return None

#     results_df = pd.DataFrame(results)
#     save_results_to_csv(results_df, shared_folder_path, "hyperopt_optimization_results.csv")

#     if trials.best_trial:
#         best_trial = trials.best_trial['result']
#         best_params = best_trial['params']
#         best_loss = best_trial['loss']
#         return best_params, -best_loss
#     else:
#         print("Не удалось найти лучшие параметры.")
#         return None

In [ ]:
# import pandas as pd
# from datetime import datetime
# from hyperopt import fmin, tpe, hp, Trials
# import backtesting


# def analyze_monthly_results(trades, intervals, initial_capital):
#     trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
#     trades.set_index('Close Time', inplace=True)
#     trades['Trade Duration'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds()

#     monthly_results = {}
#     total_pnl = 0
#     profit_months = 0
#     loss_months = 0
#     total_trade_duration = 0
#     total_trades = len(trades)

#     for start, end in intervals:
#         start_date = datetime.strptime(start, "%Y-%m")
#         end_date = datetime.strptime(end, "%Y-%m")

#         # Фильтруем сделки для текущего месяца
#         monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

#         if monthly_trades.empty:
#             monthly_results[start] = (0.0, 0)
#             continue

#         # Вычисляем PnL за месяц
#         monthly_pnl = monthly_trades['PnL'].sum() / initial_capital * 100

#         # Среднее время сделки
#         avg_trade_duration = monthly_trades['Trade Duration'].mean() / 3600

#         monthly_results[start] = (round(monthly_pnl, 2), round(avg_trade_duration, 2))

#         total_pnl += monthly_pnl
#         total_trade_duration += monthly_trades['Trade Duration'].sum()
#         if monthly_pnl > 0:
#             profit_months += 1
#         else:
#             loss_months += 1

#     avg_trade_duration_total = (total_trade_duration / total_trades) / 3600 if total_trades > 0 else 0

#     # Вычисляем процентное соотношение профитных месяцев
#     total_months = profit_months + loss_months
#     profit_month_ratio = (profit_months / total_months * 100) if total_months > 0 else 0

#     return total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration_total, profit_month_ratio


# def optimize_strategy_without_min_space(data, symbol, interval, start_date, end_date, shared_folder_path, max_evals=10):
#     intervals = generate_monthly_intervals(start_date, end_date)
#     trials = Trials()

#     # Пространство поиска
#     space = {
#         'window_length': hp.quniform('window_length', 10, 100, 2),
#         'sloping_atr_length': hp.quniform('sloping_atr_length', 7, 30, 1),
#         'take_profit_multiplier': hp.uniform('take_profit_multiplier', 1.0, 3.1),
#         'stop_loss_multiplier': hp.uniform('stop_loss_multiplier', 0.5, 1.5)
#     }

#     fixed_min_space = 5
#     results = []

#     print(f"Общее количество строк в загруженных данных: {len(data)}")
#     print(f"Диапазон дат: {data.index.min()} - {data.index.max()}")
#     print(f"Оптимизация для {symbol} ({interval}) с {start_date} по {end_date} началась...\n")

#     def objective(params):
#         SlopingStrategy.window_length = int(params['window_length'])
#         SlopingStrategy.min_space = fixed_min_space
#         SlopingStrategy.sloping_atr_length = int(params['sloping_atr_length'])
#         SlopingStrategy.take_profit_multiplier = params['take_profit_multiplier']
#         SlopingStrategy.stop_loss_multiplier = params['stop_loss_multiplier']

#         try:
#             trades, res = full_period_backtest(
#                 strategy=SlopingStrategy,
#                 data=data,
#                 initial_capital=1_000_000
#             )

#             total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration, profit_month_ratio = analyze_monthly_results(
#                 trades=trades,
#                 intervals=intervals,
#                 initial_capital=1_000_000
#             )

#             print(f"Параметры: {params}")
#             print(f"Общий профит: {total_pnl:.2f}%, Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}, Процент профитных месяцев: {profit_month_ratio:.2f}%")
#             print(f"Результаты по месяцам: {[(month, pnl) for month, (pnl, _) in monthly_results.items()]}\n")

#             row = {
#                 'symbol': symbol,
#                 'interval': interval,
#                 'window_length': int(params['window_length']),
#                 'min_space': fixed_min_space,
#                 'sloping_atr_length': int(params['sloping_atr_length']),
#                 'take_profit_multiplier': params['take_profit_multiplier'],
#                 'stop_loss_multiplier': params['stop_loss_multiplier'],
#                 'total_pnl%': total_pnl,
#                 'avg_trade_duration_hours': avg_trade_duration,
#                 'profit_month_count': profit_months,
#                 'loss_month_count': loss_months,
#                 'profit_month_ratio%': profit_month_ratio
#             }

#             for month, (pnl, duration) in monthly_results.items():
#                 row[month] = pnl

#             results.append(row)
#             return {'loss': -total_pnl, 'status': 'ok', 'params': params}

#         except Exception as e:
#             print(f"Ошибка в objective: {e}")
#             return {'loss': float('inf'), 'status': 'fail', 'params': params}

#     try:
#         fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
#     except Exception as e:
#         print(f"Ошибка при выполнении fmin: {e}")
#         return None

#     results_df = pd.DataFrame(results)
#     save_results_to_csv(results_df, shared_folder_path, "hyperopt_optimization_results.csv")

#     # Отфильтровать результаты с учетом профитных месяцев
#     filtered_results = results_df[results_df['profit_month_ratio%'] > 50]
#     filtered_results.to_csv(f"{shared_folder_path}/filtered_results.csv", index=False)

#     if trials.best_trial:
#         best_trial = trials.best_trial['result']
#         best_params = best_trial['params']
#         best_loss = best_trial['loss']
#         return best_params, -best_loss
#     else:
#         print("Не удалось найти лучшие параметры.")
#         return None


#### Check

In [ ]:
# import pandas as pd
# from datetime import datetime
# from hyperopt import fmin, tpe, hp, Trials
# import backtesting

# # Генерация интервалов
# def generate_monthly_intervals(start_date, end_date):
#     start = datetime.strptime(start_date, "%Y-%m")
#     end = datetime.strptime(end_date, "%Y-%m")
#     intervals = []
#     while start <= end:
#         next_month = datetime(start.year + (start.month // 12), (start.month % 12) + 1, 1)
#         intervals.append((start.strftime("%Y-%m"), next_month.strftime("%Y-%m")))
#         start = next_month
#     return intervals

# # Анализ помесячно
# def analyze_monthly_results(trades, intervals, initial_capital):
#     trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
#     trades.set_index('Close Time', inplace=True)
#     trades['Trade Duration'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds()

#     monthly_results = {}
#     total_pnl = 0
#     profit_months = 0
#     loss_months = 0
#     total_trade_duration = 0
#     total_trades = len(trades)

#     for start, end in intervals:
#         start_date = datetime.strptime(start, "%Y-%m")
#         end_date = datetime.strptime(end, "%Y-%m")

#         # Фильтруем сделки для текущего месяца
#         monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

#         if monthly_trades.empty:
#             monthly_results[start] = (0.0, 0)
#             continue

#         # Вычисляем PnL за месяц
#         monthly_pnl = monthly_trades['PnL'].sum() / initial_capital * 100

#         # Среднее время сделки
#         avg_trade_duration = monthly_trades['Trade Duration'].mean() / 3600

#         monthly_results[start] = (round(monthly_pnl, 2), round(avg_trade_duration, 2))

#         total_pnl += monthly_pnl
#         total_trade_duration += monthly_trades['Trade Duration'].sum()
#         if monthly_pnl > 0:
#             profit_months += 1
#         else:
#             loss_months += 1

#     avg_trade_duration_total = (total_trade_duration / total_trades) / 3600 if total_trades > 0 else 0
#     return total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration_total

# # Полный бэктест
# def full_period_backtest(strategy, data, initial_capital=1_000_000, commission=0.0005):
#     data.index = pd.to_datetime(data.index).tz_localize(None)
#     test = backtesting.Backtest(data, strategy, cash=initial_capital, commission=commission)
#     res = test.run()
#     trades = res._trades.copy()
#     return trades, res


# # Оптимизация стратегии
# def optimize_strategy(data, symbol, interval, start_date, end_date, shared_folder_path, max_evals=10):
#     intervals = generate_monthly_intervals(start_date, end_date)
#     trials = Trials()

#     space = {
#         'window_length': hp.quniform('window_length', 10, 100, 2),
#         'min_space': hp.quniform('min_space', 3, 20, 1),
#         'sloping_atr_length': hp.quniform('sloping_atr_length', 7, 30, 1),
#         'take_profit_multiplier': hp.uniform('take_profit_multiplier', 1.0, 2.5),
#         'stop_loss_multiplier': hp.uniform('stop_loss_multiplier', 1.0, 2.5)
#     }

#     results = []

#     print(f"Общее количество строк в загруженных данных: {len(data)}")
#     print(f"Диапазон дат: {data.index.min()} - {data.index.max()}")
#     print(f"Оптимизация для {symbol} ({interval}) с {start_date} по {end_date} началась...\n")

#     def objective(params):
#         SlopingStrategy.window_length = int(params['window_length'])
#         SlopingStrategy.min_space = int(params['min_space'])
#         SlopingStrategy.sloping_atr_length = int(params['sloping_atr_length'])
#         SlopingStrategy.take_profit_multiplier = params['take_profit_multiplier']
#         SlopingStrategy.stop_loss_multiplier = params['stop_loss_multiplier']

#         try:
#             trades, res = full_period_backtest(
#                 strategy=SlopingStrategy,
#                 data=data,
#                 initial_capital=1_000_000
#             )

#             total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration = analyze_monthly_results(
#                 trades=trades,
#                 intervals=intervals,
#                 initial_capital=1_000_000
#             )

#             print(f"Параметры: {params}")
#             print(f"Общий профит: {total_pnl:.2f}%, Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}")
#             print(f"Результаты по месяцам: {[(month, pnl) for month, (pnl, _) in monthly_results.items()]}\n")

#             # Формируем результат с правильным порядком колонок
#             row = {
#                 'symbol': symbol,
#                 'interval': interval,
#                 'window_length': int(params['window_length']),
#                 'min_space': int(params['min_space']),
#                 'sloping_atr_length': int(params['sloping_atr_length']),
#                 'take_profit_multiplier': params['take_profit_multiplier'],
#                 'stop_loss_multiplier': params['stop_loss_multiplier'],
#                 'total_pnl%': total_pnl,
#                 'avg_trade_duration_hours': avg_trade_duration,
#                 'profit_month_count': profit_months,
#                 'loss_month_count': loss_months
#             }

#             # Добавляем результаты по месяцам
#             for month, (pnl, duration) in monthly_results.items():
#                 row[month] = pnl

#             results.append(row)
#             return {'loss': -total_pnl, 'status': 'ok', 'params': params}

#         except Exception as e:
#             print(f"Ошибка в objective: {e}")
#             return {'loss': float('inf'), 'status': 'fail', 'params': params}
#     try:
#         fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
#     except Exception as e:
#         print(f"Ошибка при выполнении fmin: {e}")
#         return None

#     results_df = pd.DataFrame(results)
#     save_results_to_csv(results_df, shared_folder_path, "hyperopt_optimization_results.csv")

#     if trials.best_trial:
#         best_trial = trials.best_trial['result']
#         best_params = best_trial['params']
#         best_loss = best_trial['loss']
#         return best_params, -best_loss
#     else:
#         print("Не удалось найти лучшие параметры.")
#         return None

In [ ]:
import pandas as pd
from datetime import datetime
import backtesting






def check_monthly_results(trades, intervals, initial_capital):
    """
    Анализирует результаты бэктеста по месяцам.
    """
    if trades.empty:
        print("Нет сделок для анализа.")
        return 0.0, 0, 0, {}, 0.0

    # Преобразуем временные индексы
    trades['Close Time'] = pd.to_datetime(trades['ExitTime']).dt.tz_localize(None)
    trades['Entry Time'] = pd.to_datetime(trades['EntryTime']).dt.tz_localize(None)
    trades.set_index('Close Time', inplace=True)

    # Инициализация переменных
    monthly_results = {}
    total_pnl = 0.0
    profit_months = 0
    loss_months = 0
    total_trade_duration = 0
    total_trades = len(trades)

    for start, end in intervals:
        start_date = pd.to_datetime(start)
        end_date = pd.to_datetime(end)

        # Отбор сделок за месяц
        monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

        if monthly_trades.empty:
            monthly_results[start] = {'pnl': 0.0, 'trade_count': 0}
            continue

        # Рассчитываем метрики
        monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')
        monthly_pnl = monthly_trades['PnL'].sum() / initial_capital * 100
        trade_count = len(monthly_trades)
        total_trade_duration += (monthly_trades['ExitTime'] - monthly_trades['EntryTime']).dt.total_seconds().sum()

        monthly_results[start] = {'pnl': round(monthly_pnl, 2), 'trade_count': trade_count}

        total_pnl += monthly_pnl
        if monthly_pnl > 0:
            profit_months += 1
        else:
            loss_months += 1

    avg_trade_duration = total_trade_duration / total_trades / 3600 if total_trades > 0 else 0

    return total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration


def manual_backtest(data, strategy, symbol, interval, start_date, end_date, initial_capital=1_000_000,
                    window_length=32, min_space=5, sloping_atr_length=8,
                    take_profit_multiplier=1.57, stop_loss_multiplier=1.4):
    """
    Проводит бэктест стратегии с заданными параметрами и анализирует результаты по месяцам.
    """
    # Установка параметров стратегии
    strategy.window_length = window_length
    strategy.min_space = min_space
    strategy.sloping_atr_length = sloping_atr_length
    strategy.take_profit_multiplier = take_profit_multiplier
    strategy.stop_loss_multiplier = stop_loss_multiplier

    # Генерация интервалов
    intervals = generate_monthly_intervals(start_date, end_date)

    # Запуск бэктеста
    trades, res = full_period_backtest(strategy, data, initial_capital)

    # Анализ результатов
    total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration = check_monthly_results(
        trades, intervals, initial_capital
    )

    # Дополнительные метрики
    monthly_pnls = [result['pnl'] for result in monthly_results.values()]
    avg_monthly_profit = round(sum(pnl for pnl in monthly_pnls if pnl > 0) / max(profit_months, 1), 2)
    avg_monthly_loss = round(sum(pnl for pnl in monthly_pnls if pnl < 0) / max(loss_months, 1), 2)
    max_monthly_profit = max(monthly_pnls, default=0)
    min_monthly_profit = min(monthly_pnls, default=0)
    total_trades = len(trades)
    successful_trades = len(trades[trades['PnL'] > 0]) if not trades.empty else 0
    winrate = round((successful_trades / total_trades) * 100, 2) if total_trades > 0 else 0.0
    monthly_winrate = round((profit_months / (profit_months + loss_months)) * 100, 2) if (profit_months + loss_months) > 0 else 0.0

    # Итоговый результат
    final_results = {
        'symbol': symbol,
        'interval': interval,
        'window_length': window_length,
        'min_space': min_space,
        'sloping_atr_length': sloping_atr_length,
        'take_profit_multiplier': take_profit_multiplier,
        'stop_loss_multiplier': stop_loss_multiplier,
        'total_pnl%': total_pnl,
        'avg_trade_duration_hours': avg_trade_duration,
        'profit_month_count': profit_months,
        'loss_month_count': loss_months,
        'avg_monthly_profit%': avg_monthly_profit,
        'avg_monthly_loss%': avg_monthly_loss,
        'max_monthly_profit%': max_monthly_profit,
        'min_monthly_profit%': min_monthly_profit,
        'overall_winrate%': winrate,
        'monthly_winrate%': monthly_winrate
    }

    print(f"\nРезультаты бэктеста для {symbol} ({interval}):")
    print(f"Общее количество сделок: {total_trades}")
    print(f"Общий профит: {total_pnl:.2f}%, Средняя длительность сделки: {avg_trade_duration:.2f} ч.")
    print(f"Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}")
    print(f"Средний профит в месяц: {avg_monthly_profit:.2f}%, Средний убыток в месяц: {avg_monthly_loss:.2f}%")
    print(f"Максимальный профит в месяц: {max_monthly_profit:.2f}%, Минимальный профит в месяц: {min_monthly_profit:.2f}%")
    print(f"Общий winrate: {winrate:.2f}%, Winrate по месяцам: {monthly_winrate:.2f}%\n")

    print("Анализ по месяцам:")
    for month, result in monthly_results.items():
        print(f"{month}: PnL = {result['pnl']:.2f}%, Сделок = {result['trade_count']}")

    return final_results






## Backtesting and Optimization

#### 1

In [ ]:
# Настройка и вызов оптимизации
# symbol = 'SUIUSDT'
# interval = '1h'
# start_date = '2024-11'
# end_date = '2024-11'

symbol = symbol_1
interval = interval_1
start_date = start_date_1
end_date = end_date_1
max_evals = max_evals_1
params_to_optimize=params_to_optimize_1
fixed_params=fixed_params_1


# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)
print(symbol, interval)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")

SUIUSDT 15m
Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-12-31 23:45:00+00:00


In [ ]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")

# Тестируем стратегию
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()

print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)
print('Window length', SlopingStrategy.window_length)
print('Min space', SlopingStrategy.min_space)
print('Sloping atr length', SlopingStrategy.sloping_atr_length)
print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-12-31 23:45:00+00:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-12 началась...

Параметры: {'sloping_atr_length': 34.0, 'stop_loss_multiplier': 2.957853235927488, 'take_profit_multiplier': 1.7101180498145154, 'window_length': 20.0}, {'min_space': 5}
Общий профит: 43.32%, Профитные месяцы: 9, Убыточные месяцы: 6, Процент профитных месяцев: 60.00%
Результаты по месяцам: [('2023-10', 15.19), ('2023-11', 28.81), ('2023-12', 5.48), ('2024-01', 17.42), ('2024-02', 26.07), ('2024-03', -27.44), ('2024-04', -57.53), ('2024-05', 12.29), ('2024-06', -17.35), ('2024-07', -13.08), ('2024-08', -17.04), ('2024-09', 0.18), ('2024-10', 20.41), ('2024-11', 57.82), ('2024-12', -7.92)]

Параметры: {'sloping_atr_length': 26.0, 'stop_loss_multiplier': 2.946891731937408, 'take_profit_multiplier': 1.088244842096514, 'window_length': 64.0}, {'min_space': 5}
Общий профит: -35.78%, Профитные месяцы: 5, Убыточны

In [ ]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")

# Тестируем стратегию
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()

print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)
print('Window length', SlopingStrategy.window_length)
print('Min space', SlopingStrategy.min_space)
print('Sloping atr length', SlopingStrategy.sloping_atr_length)
print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00 - 2024-12-31 23:45:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-12 началась...

Параметры: {'sloping_atr_length': 28.0, 'stop_loss_multiplier': 0.6455587471108435, 'take_profit_multiplier': 2.242923541245662, 'window_length': 58.0}, {'min_space': 5}
Общий профит: -56.99%, Профитные месяцы: 7, Убыточные месяцы: 8, Процент профитных месяцев: 46.67%
Результаты по месяцам: [('2023-10', -14.65), ('2023-11', 1.01), ('2023-12', 2.16), ('2024-01', 3.01), ('2024-02', -5.54), ('2024-03', 1.38), ('2024-04', -20.12), ('2024-05', -4.27), ('2024-06', 3.99), ('2024-07', -10.18), ('2024-08', 0.8), ('2024-09', -3.09), ('2024-10', 3.29), ('2024-11', -11.87), ('2024-12', -2.91)]

Параметры: {'sloping_atr_length': 13.0, 'stop_loss_multiplier': 0.7372775695350138, 'take_profit_multiplier': 1.9571812237453314, 'window_length': 56.0}, {'min_space': 5}
Общий профит: -47.53%, Профитные месяцы: 7, Убыточные месяцы: 8, Про

SUIUSDT 30m Параметры: {'sloping_atr_length': 8.0, 'stop_loss_multiplier': 1.9156001509053375, 'take_profit_multiplier': 1.7823393677956039, 'window_length': 14.0}, {'min_space': 5}
Общий профит: 465.19%, Профитные месяцы: 10, Убыточные месяцы: 4, Процент профитных месяцев: 71.43%
Результаты по месяцам: [('2023-10', 26.7), ('2023-11', 36.67), ('2023-12', 57.85), ('2024-01', -11.77), ('2024-02', 55.62), ('2024-03', 10.25), ('2024-04', -62.01), ('2024-05', 19.25), ('2024-06', 32.65), ('2024-07', -4.16), ('2024-08', -60.33), ('2024-09', 44.34), ('2024-10', 34.82), ('2024-11', 285.3)]


In [ ]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")

# Тестируем стратегию
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()

print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)
print('Window length', SlopingStrategy.window_length)
print('Min space', SlopingStrategy.min_space)
print('Sloping atr length', SlopingStrategy.sloping_atr_length)
print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00 - 2024-12-31 23:45:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-12 началась...

Параметры: {'sloping_atr_length': 20.0, 'stop_loss_multiplier': 1.0693369696543549, 'take_profit_multiplier': 1.5600625554417626, 'window_length': 31.0}, {'min_space': 5}
Общий профит: 20.36%, Профитные месяцы: 6, Убыточные месяцы: 9, Процент профитных месяцев: 40.00%
Результаты по месяцам: [('2023-10', -10.75), ('2023-11', 24.0), ('2023-12', -7.4), ('2024-01', -2.6), ('2024-02', 11.57), ('2024-03', -17.37), ('2024-04', -6.99), ('2024-05', -5.53), ('2024-06', 4.51), ('2024-07', -6.25), ('2024-08', 8.55), ('2024-09', 43.64), ('2024-10', -1.42), ('2024-11', 16.45), ('2024-12', -30.05)]

Параметры: {'sloping_atr_length': 27.0, 'stop_loss_multiplier': 2.221100716892785, 'take_profit_multiplier': 3.309454570994015, 'window_length': 22.0}, {'min_space': 5}
Общий профит: 88.22%, Профитные месяцы: 9, Убыточные месяцы: 6, Проц

#### 2

In [ ]:
# Настройка и вызов оптимизации
# symbol = 'SUIUSDT'
# interval = '1h'
# start_date = '2024-11'
# end_date = '2024-11'

symbol = symbol_2
interval = interval_2
start_date = start_date_2
end_date = end_date_2
max_evals = max_evals_2
params_to_optimize=params_to_optimize_2
fixed_params=fixed_params_2


# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)
print(symbol, interval)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")

SUIUSDT 15m
Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-12-31 23:45:00+00:00


In [ ]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")

# Тестируем стратегию
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()

print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)
print('Window length', SlopingStrategy.window_length)
print('Min space', SlopingStrategy.min_space)
print('Sloping atr length', SlopingStrategy.sloping_atr_length)
print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-12-31 23:45:00+00:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-12 началась...

Параметры: {'sloping_atr_length': 18.0, 'stop_loss_multiplier': 1.0692758484754659, 'take_profit_multiplier': 2.5219689852060037, 'window_length': 27.0}, {'min_space': 5}
Общий профит: 3.30%, Профитные месяцы: 6, Убыточные месяцы: 9, Процент профитных месяцев: 40.00%
Результаты по месяцам: [('2023-10', -11.21), ('2023-11', 30.28), ('2023-12', -12.73), ('2024-01', -8.84), ('2024-02', 2.88), ('2024-03', -12.21), ('2024-04', -13.44), ('2024-05', -8.98), ('2024-06', 6.01), ('2024-07', -2.73), ('2024-08', -2.28), ('2024-09', 22.65), ('2024-10', -5.7), ('2024-11', 7.85), ('2024-12', 11.77)]

Параметры: {'sloping_atr_length': 10.0, 'stop_loss_multiplier': 0.9242230772737468, 'take_profit_multiplier': 2.583283864392582, 'window_length': 25.0}, {'min_space': 5}
Общий профит: -28.63%, Профитные месяцы: 6, Убыточные 

In [ ]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")

# Тестируем стратегию
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()

print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)
print('Window length', SlopingStrategy.window_length)
print('Min space', SlopingStrategy.min_space)
print('Sloping atr length', SlopingStrategy.sloping_atr_length)
print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00 - 2024-12-31 23:45:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-12 началась...

Параметры: {'sloping_atr_length': 14.0, 'stop_loss_multiplier': 1.9551353935633475, 'take_profit_multiplier': 1.3432170367815242, 'window_length': 33.0}, {'min_space': 5}
Общий профит: -21.02%, Профитные месяцы: 6, Убыточные месяцы: 9, Процент профитных месяцев: 40.00%
Результаты по месяцам: [('2023-10', 0.45), ('2023-11', -3.06), ('2023-12', -25.84), ('2024-01', -9.0), ('2024-02', 8.54), ('2024-03', 16.14), ('2024-04', -16.24), ('2024-05', -10.69), ('2024-06', -5.23), ('2024-07', -4.36), ('2024-08', 6.17), ('2024-09', 41.12), ('2024-10', -22.41), ('2024-11', 10.69), ('2024-12', -7.32)]

Параметры: {'sloping_atr_length': 14.0, 'stop_loss_multiplier': 0.9560687134801147, 'take_profit_multiplier': 2.5656087139189516, 'window_length': 70.0}, {'min_space': 5}
Общий профит: -43.94%, Профитные месяцы: 7, Убыточные месяцы: 8

SUIUSDT 30m Параметры: {'sloping_atr_length': 10.0, 'stop_loss_multiplier': 1.486332376538037, 'take_profit_multiplier': 2.6987729977712087, 'window_length': 11.0}, {'min_space': 5}
Общий профит: 1680.69%, Профитные месяцы: 11, Убыточные месяцы: 3, Процент профитных месяцев: 78.57%
Результаты по месяцам: [('2023-10', 27.05), ('2023-11', 7.76), ('2023-12', 62.65), ('2024-01', 122.69), ('2024-02', 192.15), ('2024-03', 75.34), ('2024-04', -65.41), ('2024-05', 166.11), ('2024-06', -14.83), ('2024-07', 132.57), ('2024-08', -36.7), ('2024-09', 128.78), ('2024-10', 228.34), ('2024-11', 654.18)]

In [ ]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")

# Тестируем стратегию
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()

print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)
print('Window length', SlopingStrategy.window_length)
print('Min space', SlopingStrategy.min_space)
print('Sloping atr length', SlopingStrategy.sloping_atr_length)
print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00 - 2024-12-31 23:45:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-12 началась...

Параметры: {'sloping_atr_length': 12.0, 'stop_loss_multiplier': 2.031599753590259, 'take_profit_multiplier': 1.1250662400308378, 'window_length': 52.0}, {'min_space': 5}
Общий профит: -50.79%, Профитные месяцы: 7, Убыточные месяцы: 8, Процент профитных месяцев: 46.67%
Результаты по месяцам: [('2023-10', -7.71), ('2023-11', 2.18), ('2023-12', 7.06), ('2024-01', -13.24), ('2024-02', 4.33), ('2024-03', 6.0), ('2024-04', -16.67), ('2024-05', -7.53), ('2024-06', 0.43), ('2024-07', -16.11), ('2024-08', 13.42), ('2024-09', 16.66), ('2024-10', -22.02), ('2024-11', -8.87), ('2024-12', -8.71)]

Параметры: {'sloping_atr_length': 19.0, 'stop_loss_multiplier': 2.3612014417354863, 'take_profit_multiplier': 3.071492362234611, 'window_length': 24.0}, {'min_space': 5}
Общий профит: -27.92%, Профитные месяцы: 6, Убыточные месяцы: 9, Пр

#### 3

In [ ]:
# Настройка и вызов оптимизации
# symbol = 'SUIUSDT'
# interval = '1h'
# start_date = '2024-11'
# end_date = '2024-11'

symbol = symbol_3
interval = interval_3
start_date = start_date_3
end_date = end_date_3
max_evals = max_evals_3
params_to_optimize=params_to_optimize_3
fixed_params=fixed_params_3


# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)
print(symbol, interval)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")

SUIUSDT 15m
Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-12-31 23:45:00+00:00


In [ ]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")

# Тестируем стратегию
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()

print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)
print('Window length', SlopingStrategy.window_length)
print('Min space', SlopingStrategy.min_space)
print('Sloping atr length', SlopingStrategy.sloping_atr_length)
print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

Общее количество строк в загруженных данных: 43968
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-12-31 23:45:00+00:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-12 началась...

Параметры: {'sloping_atr_length': 7.0, 'stop_loss_multiplier': 1.8071454751212965, 'take_profit_multiplier': 2.0895474473906814, 'window_length': 70.0}, {'min_space': 5}
Общий профит: -31.99%, Профитные месяцы: 4, Убыточные месяцы: 11, Процент профитных месяцев: 26.67%
Результаты по месяцам: [('2023-10', -3.79), ('2023-11', -1.48), ('2023-12', -11.61), ('2024-01', -20.43), ('2024-02', 4.46), ('2024-03', 6.26), ('2024-04', -8.15), ('2024-05', -16.11), ('2024-06', -1.84), ('2024-07', -0.28), ('2024-08', -6.08), ('2024-09', 11.15), ('2024-10', -0.4), ('2024-11', -5.79), ('2024-12', 22.09)]

Параметры: {'sloping_atr_length': 37.0, 'stop_loss_multiplier': 2.1458432895849118, 'take_profit_multiplier': 2.478579814391659, 'window_length': 22.0}, {'min_space': 5}
Общий профит: 213.19%, Профитные месяцы: 10, Убыточн

In [ ]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")

# Тестируем стратегию
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()

print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)
print('Window length', SlopingStrategy.window_length)
print('Min space', SlopingStrategy.min_space)
print('Sloping atr length', SlopingStrategy.sloping_atr_length)
print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

SUIUSDT 30m Параметры: {'sloping_atr_length': 13.0, 'stop_loss_multiplier': 1.2906831921295494, 'take_profit_multiplier': 2.9991697641252095, 'window_length': 11.0}, {'min_space': 5}
Общий профит: 1463.56%, Профитные месяцы: 11, Убыточные месяцы: 3, Процент профитных месяцев: 78.57%
Результаты по месяцам: [('2023-10', 33.13), ('2023-11', 32.03), ('2023-12', 79.51), ('2024-01', 76.62), ('2024-02', 189.55), ('2024-03', 214.55), ('2024-04', -90.42), ('2024-05', 15.02), ('2024-06', -46.91), ('2024-07', 71.29), ('2024-08', 74.28), ('2024-09', -81.18), ('2024-10', 409.32), ('2024-11', 486.77)]


In [ ]:
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path,
    params_to_optimize=params_to_optimize,
    fixed_params=fixed_params
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = fixed_params.get('window_length') or int(best_params['window_length'])
SlopingStrategy.min_space = fixed_params.get('min_space') or int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = fixed_params.get('sloping_atr_length') or int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = fixed_params.get('take_profit_multiplier') or best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = fixed_params.get('stop_loss_multiplier') or best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")

# Тестируем стратегию
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()

print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)
print('Window length', SlopingStrategy.window_length)
print('Min space', SlopingStrategy.min_space)
print('Sloping atr length', SlopingStrategy.sloping_atr_length)
print('Take profit multiplier', SlopingStrategy.take_profit_multiplier)
print('Stop loss multiplier', SlopingStrategy.stop_loss_multiplier)

SUIUSDT 30m Параметры: {'sloping_atr_length': 14.0, 'stop_loss_multiplier': 1.5469231129828445, 'take_profit_multiplier': 2.975362191464476, 'window_length': 11.0}, {'min_space': 5}
Общий профит: 2139.17%, Профитные месяцы: 13, Убыточные месяцы: 1, Процент профитных месяцев: 92.86%
Результаты по месяцам: [('2023-10', 34.61), ('2023-11', 13.22), ('2023-12', 104.05), ('2024-01', 137.54), ('2024-02', 216.52), ('2024-03', 177.63), ('2024-04', -137.0), ('2024-05', 51.27), ('2024-06', 17.87), ('2024-07', 120.34), ('2024-08', 22.41), ('2024-09', 69.52), ('2024-10', 352.85), ('2024-11', 958.34)]


## Final analysis

### Optimization results

#### full optimization

In [ ]:
import pandas as pd

# Считаем данные из файла
file_path = '/content/drive/MyDrive/Sloping bot/optimized_params.csv'
df_optimized = pd.read_csv(file_path)
df_optimized

,Symbol,Interval,Start Date,End Date,Test Return [%],window_length,min_space,sloping_atr_length,stop_loss_multiplier,take_profit_multiplier
0,NOTUSDT,30m,2024-05,2024-10,233.572465,80.0,NaN,17.0,3.471899,3.984025
1,NOTUSDT,30m,2024-05,2024-10,103.217958,14.0,NaN,20.0,1.086321,2.724040
2,NOTUSDT,30m,2024-05,2024-10,-38.082432,24.0,NaN,11.0,3.325552,3.461578
3,NOTUSDT,30m,2024-05,2024-10,24.518548,48.0,NaN,29.0,1.894256,1.208675
4,NOTUSDT,30m,2024-05,2024-10,77.741279,58.0,NaN,19.0,3.176453,2.906685
...,...,...,...,...,...,...,...,...,...,...
3032,APEUSDT,4h,2022-06,2024-10,-43.079953,52.0,10.0,21.0,1.500000,1.500000
3033,APEUSDT,4h,2022-06,2024-10,210.131328,22.0,18.0,17.0,1.500000,1.500000
3034,APEUSDT,4h,2022-06,2024-10,232.527972,38.0,18.0,33.0,1.500000,1.500000
3035,APEUSDT,4h,2022-06,2024-10,-3.252550,30.0,19.0,28.0,1.500000,1.500000


In [ ]:
df_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3037 entries, 0 to 3036
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Symbol                  3037 non-null   object 
 1   Interval                3037 non-null   object 
 2   Start Date              3037 non-null   object 
 3   End Date                3037 non-null   object 
 4   Test Return [%]         3037 non-null   float64
 5   window_length           3037 non-null   float64
 6   min_space               178 non-null    float64
 7   sloping_atr_length      3037 non-null   float64
 8   stop_loss_multiplier    3035 non-null   float64
 9   take_profit_multiplier  3035 non-null   float64
dtypes: float64(6), object(4)
memory usage: 237.4+ KB


In [ ]:
file_name = "optimized_params.csv"
df_optimized.to_csv(file_name, index=False)

In [ ]:
# Удалим дубликаты, если они есть
df_optimized.drop_duplicates(inplace=True)
df_optimized.reset_index(drop=True, inplace=True)
df_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3037 entries, 0 to 3036
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Symbol                  3037 non-null   object 
 1   Interval                3037 non-null   object 
 2   Start Date              3037 non-null   object 
 3   End Date                3037 non-null   object 
 4   Test Return [%]         3037 non-null   float64
 5   window_length           3037 non-null   float64
 6   min_space               178 non-null    float64
 7   sloping_atr_length      3037 non-null   float64
 8   stop_loss_multiplier    3035 non-null   float64
 9   take_profit_multiplier  3035 non-null   float64
dtypes: float64(6), object(4)
memory usage: 237.4+ KB


In [ ]:
df_optimized

,Symbol,Interval,Start Date,End Date,Test Return [%],window_length,min_space,sloping_atr_length,stop_loss_multiplier,take_profit_multiplier
0,NOTUSDT,30m,2024-05,2024-10,233.572465,80.0,NaN,17.0,3.471899,3.984025
1,NOTUSDT,30m,2024-05,2024-10,103.217958,14.0,NaN,20.0,1.086321,2.724040
2,NOTUSDT,30m,2024-05,2024-10,-38.082432,24.0,NaN,11.0,3.325552,3.461578
3,NOTUSDT,30m,2024-05,2024-10,24.518548,48.0,NaN,29.0,1.894256,1.208675
4,NOTUSDT,30m,2024-05,2024-10,77.741279,58.0,NaN,19.0,3.176453,2.906685
...,...,...,...,...,...,...,...,...,...,...
3032,APEUSDT,4h,2022-06,2024-10,-43.079953,52.0,10.0,21.0,1.500000,1.500000
3033,APEUSDT,4h,2022-06,2024-10,210.131328,22.0,18.0,17.0,1.500000,1.500000
3034,APEUSDT,4h,2022-06,2024-10,232.527972,38.0,18.0,33.0,1.500000,1.500000
3035,APEUSDT,4h,2022-06,2024-10,-3.252550,30.0,19.0,28.0,1.500000,1.500000


In [ ]:
coin_dfs = {}

symbols = df_optimized['Symbol'].unique()

for symbol in symbols:
    # Filter data by symbol
    coin_data = df_optimized[df_optimized['Symbol'] == symbol]

    # Sort data by 'Interval' and 'Test Return [%]' in descending order
    sorted_coin_data = coin_data.sort_values(by=['Interval', 'Test Return [%]'], ascending=[True, False])

    # Store the dataframe in the dictionary using the symbol as the key
    coin_dfs[f"{symbol}_optimized_df"] = sorted_coin_data

    # Save to CSV file
    file_name = f"{symbol}_optimization_params.csv"
    sorted_coin_data.to_csv(file_name, index=False)

    print(f'File {file_name} created for download, data saved in {symbol}_optimized_df.')

File NOTUSDT_optimization_params.csv created for download, data saved in NOTUSDT_optimized_df.
File WLDUSDT_optimization_params.csv created for download, data saved in WLDUSDT_optimized_df.
File TIAUSDT_optimization_params.csv created for download, data saved in TIAUSDT_optimized_df.
File XRPUSDT_optimization_params.csv created for download, data saved in XRPUSDT_optimized_df.
File TONUSDT_optimization_params.csv created for download, data saved in TONUSDT_optimized_df.
File SOLUSDT_optimization_params.csv created for download, data saved in SOLUSDT_optimized_df.
File APEUSDT_optimization_params.csv created for download, data saved in APEUSDT_optimized_df.
File WIFUSDT_optimization_params.csv created for download, data saved in WIFUSDT_optimized_df.
File ADAUSDT_optimization_params.csv created for download, data saved in ADAUSDT_optimized_df.
File BTCUSDT_optimization_params.csv created for download, data saved in BTCUSDT_optimized_df.
File ETHUSDT_optimization_params.csv created for d

In [ ]:
import pandas as pd

# Задаём пользовательский порядок сортировки для interval
interval_order = {'4h': 0, '1h': 1, '15m': 2, '5m': 3, '1m': 4}

# Добавляем новый столбец для пользовательской сортировки
df_filtered = df_optimized[df_optimized['Test Return [%]'] > 100].copy()
df_filtered['interval_order'] = df_filtered['Interval'].map(interval_order)

# Сортировка по символу, interval_order и total_pnl% (по убыванию)
df_sorted = df_filtered.sort_values(
    by=['Symbol', 'interval_order', 'Test Return [%]'],
    ascending=[True, True, False]
).drop(columns=['interval_order'])  # Убираем временный столбец

# Сохраняем в CSV (если нужно)
df_sorted.to_csv('sorted_data.csv', index=False, encoding='utf-8')

print("Файл успешно сохранён как sorted_data.csv")

Файл успешно сохранён как sorted_data.csv


In [ ]:
# NOTUSDT_optimized_df

In [ ]:
sorted_coin_data

,Symbol,Interval,Start Date,End Date,Test Return [%],window_length,min_space,sloping_atr_length,stop_loss_multiplier,take_profit_multiplier
1298,FILUSDT,4h,2024-09,2024-10,inf,50.0,NaN,14.0,1.5,1.5
1295,FILUSDT,4h,2024-09,2024-10,33.752706,10.0,NaN,7.0,NaN,NaN
2685,FILUSDT,4h,2024-09,2024-10,33.752706,10.0,NaN,7.0,NaN,NaN
1297,FILUSDT,4h,2024-09,2024-10,9.964792,50.0,NaN,14.0,1.5,1.5
2687,FILUSDT,4h,2024-09,2024-10,9.964792,50.0,NaN,14.0,1.5,1.5
2686,FILUSDT,4h,2024-09,2024-10,7.473594,50.0,NaN,14.0,1.5,1.5
1296,FILUSDT,4h,2024-09,2024-10,7.473594,50.0,NaN,14.0,1.5,1.5


#### hyperopt optimization by month

In [ ]:
import pandas as pd

# Считаем данные из файла
file_path_by_month = '/content/drive/MyDrive/Sloping bot/hyperopt_optimization_results.csv'
df_optimized_by_month = pd.read_csv(file_path_by_month)
df_optimized_by_month

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
0,BTCUSDT,4h,37,8.0,10,1.856871,1.252420,-65.807486,26.230366,18,...,-5.87,-3.75,-1.40,-0.83,0.21,1.26,-0.91,1.13,36.734694,NaN
1,BTCUSDT,4h,32,10.0,22,1.290515,1.469040,-57.109051,23.248677,20,...,-11.27,-0.11,-1.76,-0.91,3.01,-3.38,-2.25,1.38,40.816327,NaN
2,BTCUSDT,4h,31,14.0,21,1.663784,1.531341,-76.389729,30.827586,18,...,-5.95,0.04,-0.72,-0.36,-0.49,1.49,-0.68,0.51,36.734694,NaN
3,BTCUSDT,4h,34,10.0,29,1.133459,1.282490,-45.549995,18.742268,24,...,-9.26,0.11,-2.29,-3.93,3.99,-8.61,-2.66,1.67,48.979592,NaN
4,BTCUSDT,4h,43,11.0,15,1.396247,1.982624,-65.743008,31.259740,17,...,-0.13,0.47,0.89,-0.36,0.79,2.70,-1.91,1.05,34.693878,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89959,SUIUSDT,30m,71,3.0,19,2.076654,1.126309,-13.891202,3.084071,6,...,7.61,-6.80,-3.96,1.30,-8.53,-25.34,-5.16,-2.13,42.857143,7.28
89960,SUIUSDT,30m,36,4.0,14,2.203660,1.211254,-17.408495,3.773454,7,...,27.36,-4.10,-9.41,-3.43,2.46,-10.47,7.86,6.22,50.000000,6.95
89961,SUIUSDT,30m,56,8.0,28,1.528378,0.823253,-39.438727,1.755843,4,...,7.58,2.11,-11.07,-0.97,-7.39,-2.16,-8.74,6.03,28.571429,-2.26
89962,SUIUSDT,30m,49,5.0,12,1.658147,1.404611,-74.169979,3.115478,6,...,8.05,-9.47,-5.38,3.88,-7.32,-5.00,0.83,0.34,42.857143,1.24


In [ ]:
# df_optimized_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5842 entries, 0 to 5841
Data columns (total 60 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   symbol                    5842 non-null   object 
 1   interval                  5842 non-null   object 
 2   window_length             5842 non-null   int64  
 3   min_space                 5842 non-null   int64  
 4   sloping_atr_length        5842 non-null   int64  
 5   take_profit_multiplier    5842 non-null   float64
 6   stop_loss_multiplier      5842 non-null   float64
 7   total_pnl%                5842 non-null   float64
 8   avg_trade_duration_hours  5842 non-null   float64
 9   profit_month_count        5842 non-null   int64  
 10  loss_month_count          5842 non-null   int64  
 11  2020-10                   5142 non-null   float64
 12  2020-11                   5142 non-null   float64
 13  2020-12                   5142 non-null   float64
 14  2021-01 

In [ ]:
# Удалим дубликаты, если они есть
df_optimized_by_month.drop_duplicates(inplace=True)
df_optimized_by_month.reset_index(drop=True, inplace=True)

file_name = "hyperopt_optimization_results.csv"
df_optimized_by_month.to_csv(file_name, index=False)

df_optimized_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89964 entries, 0 to 89963
Data columns (total 62 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   symbol                    89964 non-null  object 
 1   interval                  89964 non-null  object 
 2   window_length             89964 non-null  int64  
 3   min_space                 89964 non-null  float64
 4   sloping_atr_length        89964 non-null  int64  
 5   take_profit_multiplier    89964 non-null  float64
 6   stop_loss_multiplier      89964 non-null  float64
 7   total_pnl%                89964 non-null  float64
 8   avg_trade_duration_hours  89964 non-null  float64
 9   profit_month_count        89964 non-null  int64  
 10  loss_month_count          89964 non-null  int64  
 11  2020-10                   40322 non-null  float64
 12  2020-11                   40322 non-null  float64
 13  2020-12                   40322 non-null  float64
 14  2021-0

In [ ]:
df_optimized_by_month

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
0,BTCUSDT,4h,37,8.0,10,1.856871,1.252420,-65.807486,26.230366,18,...,-5.87,-3.75,-1.40,-0.83,0.21,1.26,-0.91,1.13,36.734694,NaN
1,BTCUSDT,4h,32,10.0,22,1.290515,1.469040,-57.109051,23.248677,20,...,-11.27,-0.11,-1.76,-0.91,3.01,-3.38,-2.25,1.38,40.816327,NaN
2,BTCUSDT,4h,31,14.0,21,1.663784,1.531341,-76.389729,30.827586,18,...,-5.95,0.04,-0.72,-0.36,-0.49,1.49,-0.68,0.51,36.734694,NaN
3,BTCUSDT,4h,34,10.0,29,1.133459,1.282490,-45.549995,18.742268,24,...,-9.26,0.11,-2.29,-3.93,3.99,-8.61,-2.66,1.67,48.979592,NaN
4,BTCUSDT,4h,43,11.0,15,1.396247,1.982624,-65.743008,31.259740,17,...,-0.13,0.47,0.89,-0.36,0.79,2.70,-1.91,1.05,34.693878,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89959,SUIUSDT,30m,71,3.0,19,2.076654,1.126309,-13.891202,3.084071,6,...,7.61,-6.80,-3.96,1.30,-8.53,-25.34,-5.16,-2.13,42.857143,7.28
89960,SUIUSDT,30m,36,4.0,14,2.203660,1.211254,-17.408495,3.773454,7,...,27.36,-4.10,-9.41,-3.43,2.46,-10.47,7.86,6.22,50.000000,6.95
89961,SUIUSDT,30m,56,8.0,28,1.528378,0.823253,-39.438727,1.755843,4,...,7.58,2.11,-11.07,-0.97,-7.39,-2.16,-8.74,6.03,28.571429,-2.26
89962,SUIUSDT,30m,49,5.0,12,1.658147,1.404611,-74.169979,3.115478,6,...,8.05,-9.47,-5.38,3.88,-7.32,-5.00,0.83,0.34,42.857143,1.24


In [ ]:
import pandas as pd

# Задаём пользовательский порядок сортировки для interval
interval_order = {'4h': 0, '1h': 1, '15m': 2, '5m': 3, '1m': 4}

# Добавляем новый столбец для пользовательской сортировки
df_filtered_by_month = df_optimized_by_month[df_optimized_by_month['profit_month_ratio%'] > 70].copy()
df_filtered_by_month['interval_order'] = df_filtered_by_month['interval'].map(interval_order)

# Сортировка по символу, interval_order и total_pnl% (по убыванию)
df_sorted_by_month = df_filtered_by_month.sort_values(
    by=['symbol', 'interval_order', 'total_pnl%'],
    ascending=[True, True, False]
).drop(columns=['interval_order'])  # Убираем временный столбец

# Сохраняем в CSV (если нужно)
df_sorted_by_month.to_csv('sorted_data_by_month.csv', index=False, encoding='utf-8')

print("Файл успешно сохранён как sorted_data_by_month.csv")

Файл успешно сохранён как sorted_data_by_month.csv


In [ ]:
df_sorted_by_month

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
21736,1000PEPEUSDT,1h,98,5.0,7,1.661477,1.834137,629.728243,10.239203,12,...,172.45,-18.66,179.80,-23.53,22.95,22.06,25.40,153.57,70.588235,NaN
18891,1000PEPEUSDT,1h,92,5.0,7,1.457082,2.430666,621.659050,11.260000,14,...,178.86,-89.45,47.38,78.15,78.64,3.72,18.61,155.90,82.352941,NaN
21806,1000PEPEUSDT,1h,98,5.0,10,1.484407,2.238799,612.382192,9.550000,14,...,194.50,-66.93,89.04,18.54,18.71,22.76,26.43,160.70,82.352941,NaN
19264,1000PEPEUSDT,1h,92,5.0,8,1.089490,2.507078,612.128629,7.924699,14,...,139.73,-36.66,9.18,59.87,51.82,49.91,45.29,164.63,82.352941,NaN
18704,1000PEPEUSDT,1h,72,5.0,10,1.418003,2.246889,606.723701,9.440111,12,...,120.03,-57.45,6.45,134.64,143.95,35.89,34.10,98.77,70.588235,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52699,XRPUSDT,1h,46,5.0,22,2.533609,1.672893,0.977098,19.375000,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.98
52756,XRPUSDT,1h,32,5.0,18,2.512465,1.638538,0.894114,21.315789,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.89
52838,XRPUSDT,1h,24,5.0,28,2.404548,1.231500,0.679034,12.172414,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.68
52736,XRPUSDT,1h,32,5.0,23,2.452865,2.594793,0.580199,26.687500,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.000000,0.58


#### Check

In [ ]:
data = pd.read_excel("check1.xlsx")
data

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
0,SUIUSDT,15m,17,11,17,1.5,0.5,955.719543,4.296598,11,...,112.88,-37.29,55.58,-82.43,38.51,19.73,146.76,129.92,78.571429,544.13


In [ ]:
symbol = data['symbol'].iloc[0]
interval = data['interval'].iloc[0]
start_date = '2023-10'
end_date = '2024-11'

df = download_klines(symbol, interval, start_date, end_date)

for i in range(len(data)):
  print(i+1, '/', (len(data)))

  window_length = data['window_length'].iloc[i]
  min_space = data['min_space'].iloc[i]
  sloping_atr_length = data['sloping_atr_length'].iloc[i]
  take_profit_multiplier = data['take_profit_multiplier'].iloc[i]
  stop_loss_multiplier = data['stop_loss_multiplier'].iloc[i]

  print(f'{symbol, interval, start_date, end_date}, window_length={window_length}, min_space={min_space}, slopes_atr_length={sloping_atr_length}, take_profit_multiplier={take_profit_multiplier}, stop_loss_multiplier={stop_loss_multiplier}')
  manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
    )
    # Если нужно сохранить результаты в CSV
  manual_results_df = pd.DataFrame([manual_results])
  manual_results_df
  # manual_results_df.to_csv("manual_backtest_results.csv", index=False)


  test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
  res = test.run()
  print(symbol)
  print()
  print("Результаты с оптимизированными параметрами:", res)
  print('---' * 30)


1 / 1
('SUIUSDT', '15m', '2023-10', '2024-11'), window_length=17, min_space=11, slopes_atr_length=17, take_profit_multiplier=1.5, stop_loss_multiplier=0.5


<ipython-input-36-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для SUIUSDT (15m):
Общее количество сделок: 2876
Общий профит: -64.06%, Средняя длительность сделки: 0.50 ч.
Профитные месяцы: 2, Убыточные месяцы: 12
Средний профит в месяц: 3.66%, Средний убыток в месяц: -5.95%
Максимальный профит в месяц: 5.62%, Минимальный профит в месяц: -15.38%
Общий winrate: 25.17%, Winrate по месяцам: 14.29%

Анализ по месяцам:
2023-10: PnL = -11.69%, Сделок = 202
2023-11: PnL = -15.38%, Сделок = 202
2023-12: PnL = -14.55%, Сделок = 219
2024-01: PnL = -5.06%, Сделок = 207
2024-02: PnL = -1.71%, Сделок = 203
2024-03: PnL = -1.32%, Сделок = 210
2024-04: PnL = -2.61%, Сделок = 196
2024-05: PnL = -2.69%, Сделок = 208
2024-06: PnL = -3.20%, Сделок = 196
2024-07: PnL = -2.30%, Сделок = 203
2024-08: PnL = -6.66%, Сделок = 208
2024-09: PnL = -4.22%, Сделок = 193
2024-10: PnL = 5.62%, Сделок = 218
2024-11: PnL = 1.71%, Сделок = 211
SUIUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End                 

In [ ]:
# data.info()

In [ ]:
symbol = data['symbol'].iloc[0]
interval = data['interval'].iloc[0]
start_date = '2023-10'
end_date = '2024-11'

df = download_klines(symbol, interval, start_date, end_date)

for i in range(len(data)):
  print(i+1, '/', (len(data)))

  window_length = data['window_length'].iloc[i]
  min_space = data['min_space'].iloc[i]
  sloping_atr_length = data['sloping_atr_length'].iloc[i]
  take_profit_multiplier = data['take_profit_multiplier'].iloc[i]
  stop_loss_multiplier = data['stop_loss_multiplier'].iloc[i]

  print(f'{symbol, interval, start_date, end_date}, window_length={window_length}, min_space={min_space}, slopes_atr_length={sloping_atr_length}, take_profit_multiplier={take_profit_multiplier}, stop_loss_multiplier={stop_loss_multiplier}')
  manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
    )
    # Если нужно сохранить результаты в CSV
  manual_results_df = pd.DataFrame([manual_results])
  manual_results_df
  # manual_results_df.to_csv("manual_backtest_results.csv", index=False)


  test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
  res = test.run()
  print(symbol)
  print()
  print("Результаты с оптимизированными параметрами:", res)
  print('---' * 30)


1 / 5
('1000PEPEUSDT', '30m', '2023-10', '2024-11'), window_length=10, min_space=5, slopes_atr_length=24, take_profit_multiplier=3.04155181615615, stop_loss_multiplier=0.798885066452841


<ipython-input-32-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для 1000PEPEUSDT (30m):
Общее количество сделок: 1352
Общий профит: 955.72%, Средняя длительность сделки: 4.30 ч.
Профитные месяцы: 11, Убыточные месяцы: 3
Средний профит в месяц: 100.14%, Средний убыток в месяц: -48.61%
Максимальный профит в месяц: 544.13%, Минимальный профит в месяц: -82.43%
Общий winrate: 24.70%, Winrate по месяцам: 78.57%

Анализ по месяцам:
2023-10: PnL = 22.88%, Сделок = 89
2023-11: PnL = 16.57%, Сделок = 89
2023-12: PnL = 10.61%, Сделок = 100
2024-01: PnL = 3.98%, Сделок = 93
2024-02: PnL = -26.10%, Сделок = 103
2024-03: PnL = 112.88%, Сделок = 102
2024-04: PnL = -37.29%, Сделок = 108
2024-05: PnL = 55.58%, Сделок = 97
2024-06: PnL = -82.43%, Сделок = 98
2024-07: PnL = 38.51%, Сделок = 99
2024-08: PnL = 19.73%, Сделок = 101
2024-09: PnL = 146.76%, Сделок = 81
2024-10: PnL = 129.92%, Сделок = 93
2024-11: PnL = 544.13%, Сделок = 99
1000PEPEUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End      

<ipython-input-32-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для 1000PEPEUSDT (30m):
Общее количество сделок: 1352
Общий профит: 900.66%, Средняя длительность сделки: 4.30 ч.
Профитные месяцы: 11, Убыточные месяцы: 3
Средний профит в месяц: 93.87%, Средний убыток в месяц: -43.97%
Максимальный профит в месяц: 483.53%, Минимальный профит в месяц: -68.62%
Общий winrate: 24.70%, Winrate по месяцам: 78.57%

Анализ по месяцам:
2023-10: PnL = 22.85%, Сделок = 89
2023-11: PnL = 16.13%, Сделок = 89
2023-12: PnL = 9.99%, Сделок = 100
2024-01: PnL = 2.46%, Сделок = 94
2024-02: PnL = -26.15%, Сделок = 103
2024-03: PnL = 108.76%, Сделок = 102
2024-04: PnL = -37.14%, Сделок = 108
2024-05: PnL = 52.68%, Сделок = 97
2024-06: PnL = -68.62%, Сделок = 98
2024-07: PnL = 38.63%, Сделок = 99
2024-08: PnL = 19.23%, Сделок = 101
2024-09: PnL = 148.22%, Сделок = 81
2024-10: PnL = 130.10%, Сделок = 93
2024-11: PnL = 483.53%, Сделок = 98
1000PEPEUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End        

...


Результаты бэктеста для 1000PEPEUSDT (30m):
Общее количество сделок: 402
Общий профит: 412.59%, Средняя длительность сделки: 10.16 ч.
Профитные месяцы: 9, Убыточные месяцы: 5
Средний профит в месяц: 77.58%, Средний убыток в месяц: -57.13%
Максимальный профит в месяц: 175.29%, Минимальный профит в месяц: -205.92%
Общий winrate: 59.70%, Winrate по месяцам: 64.29%

Анализ по месяцам:
2023-10: PnL = -22.02%, Сделок = 24
2023-11: PnL = 48.53%, Сделок = 27
2023-12: PnL = -16.78%, Сделок = 29
2024-01: PnL = -4.50%, Сделок = 28
2024-02: PnL = 3.00%, Сделок = 23
2024-03: PnL = 2.13%, Сделок = 34
2024-04: PnL = 67.06%, Сделок = 30
2024-05: PnL = 31.95%, Сделок = 30
2024-06: PnL = 133.08%, Сделок = 28
2024-07: PnL = 175.29%, Сделок = 32
2024-08: PnL = 86.62%, Сделок = 26
2024-09: PnL = -36.44%, Сделок = 30
2024-10: PnL = 150.60%, Сделок = 31
2024-11: PnL = -205.92%, Сделок = 30
1000PEPEUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End            

<ipython-input-32-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для 1000PEPEUSDT (30m):
Общее количество сделок: 436
Общий профит: 578.09%, Средняя длительность сделки: 8.60 ч.
Профитные месяцы: 10, Убыточные месяцы: 4
Средний профит в месяц: 82.51%, Средний убыток в месяц: -61.74%
Максимальный профит в месяц: 227.64%, Минимальный профит в месяц: -201.17%
Общий winrate: 63.53%, Winrate по месяцам: 71.43%

Анализ по месяцам:
2023-10: PnL = -19.92%, Сделок = 27
2023-11: PnL = 39.16%, Сделок = 27
2023-12: PnL = 8.64%, Сделок = 33
2024-01: PnL = 7.81%, Сделок = 30
2024-02: PnL = -15.87%, Сделок = 28
2024-03: PnL = 5.24%, Сделок = 34
2024-04: PnL = 55.35%, Сделок = 35
2024-05: PnL = 36.61%, Сделок = 31
2024-06: PnL = 106.30%, Сделок = 34
2024-07: PnL = 183.48%, Сделок = 32
2024-08: PnL = 154.82%, Сделок = 27
2024-09: PnL = -10.00%, Сделок = 32
2024-10: PnL = 227.64%, Сделок = 35
2024-11: PnL = -201.17%, Сделок = 31
1000PEPEUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End            

<ipython-input-32-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для 1000PEPEUSDT (30m):
Общее количество сделок: 496
Общий профит: 512.36%, Средняя длительность сделки: 7.28 ч.
Профитные месяцы: 10, Убыточные месяцы: 4
Средний профит в месяц: 56.35%, Средний убыток в месяц: -12.78%
Максимальный профит в месяц: 144.34%, Минимальный профит в месяц: -26.76%
Общий winrate: 64.31%, Winrate по месяцам: 71.43%

Анализ по месяцам:
2023-10: PnL = -3.58%, Сделок = 31
2023-11: PnL = 27.76%, Сделок = 32
2023-12: PnL = -14.89%, Сделок = 40
2024-01: PnL = -5.89%, Сделок = 35
2024-02: PnL = 14.67%, Сделок = 35
2024-03: PnL = 31.77%, Сделок = 40
2024-04: PnL = 53.39%, Сделок = 35
2024-05: PnL = 8.18%, Сделок = 31
2024-06: PnL = 144.34%, Сделок = 31
2024-07: PnL = 102.98%, Сделок = 45
2024-08: PnL = 64.37%, Сделок = 29
2024-09: PnL = 60.15%, Сделок = 39
2024-10: PnL = 55.88%, Сделок = 43
2024-11: PnL = -26.76%, Сделок = 30
1000PEPEUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End                

In [ ]:
window_length = 98
min_space = 5
sloping_atr_length = 7
take_profit_multiplier = 1.661
stop_loss_multiplier = 1.834

manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
)
# Если нужно сохранить результаты в CSV
manual_results_df = pd.DataFrame([manual_results])
manual_results_df
# manual_results_df.to_csv("manual_backtest_results.csv", index=False)


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

<ipython-input-30-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для 1000PEPEUSDT (1h):
Общее количество сделок: 248
Общий профит: 367.21%, Средняя длительность сделки: 10.61 ч.
Профитные месяцы: 11, Убыточные месяцы: 3
Средний профит в месяц: 68.05%, Средний убыток в месяц: -127.12%
Максимальный профит в месяц: 197.65%, Минимальный профит в месяц: -334.90%
Общий winrate: 60.48%, Winrate по месяцам: 78.57%

Анализ по месяцам:
2023-10: PnL = 7.15%, Сделок = 18
2023-11: PnL = 16.29%, Сделок = 16
2023-12: PnL = 7.88%, Сделок = 15
2024-01: PnL = 46.17%, Сделок = 16
2024-02: PnL = 37.80%, Сделок = 12
2024-03: PnL = 189.62%, Сделок = 19
2024-04: PnL = -20.56%, Сделок = 20
2024-05: PnL = 197.65%, Сделок = 16
2024-06: PnL = -25.91%, Сделок = 21
2024-07: PnL = 25.20%, Сделок = 14
2024-08: PnL = 24.22%, Сделок = 16
2024-09: PnL = 27.88%, Сделок = 21
2024-10: PnL = 168.73%, Сделок = 22
2024-11: PnL = -334.90%, Сделок = 22
1000PEPEUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End            

('1000PEPEUSDT', '30m', '2023-10', '2024-11'), window_length=10, min_space=5, slopes_atr_length=24, take_profit_multiplier=3.04155181615615, stop_loss_multiplier=0.798885066452841
<ipython-input-32-03ecb89204fb>:43: SettingWithCopyWarning:
A value is trying to be set on a copy of a slice from a DataFrame.

In [ ]:
window_length = 10
min_space = 5
sloping_atr_length = 24
take_profit_multiplier = 3
stop_loss_multiplier = 0.8
start_date = '2023-10'
end_date = '2024-11'

manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
)
# Если нужно сохранить результаты в CSV
manual_results_df = pd.DataFrame([manual_results])
manual_results_df
# manual_results_df.to_csv("manual_backtest_results.csv", index=False)


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

<ipython-input-32-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для 1000PEPEUSDT (30m):
Общее количество сделок: 1591
Общий профит: -74.18%, Средняя длительность сделки: 3.44 ч.
Профитные месяцы: 6, Убыточные месяцы: 8
Средний профит в месяц: 4.81%, Средний убыток в месяц: -12.88%
Максимальный профит в месяц: 16.65%, Минимальный профит в месяц: -18.98%
Общий winrate: 21.50%, Winrate по месяцам: 42.86%

Анализ по месяцам:
2023-10: PnL = -17.97%, Сделок = 98
2023-11: PnL = -13.50%, Сделок = 114
2023-12: PnL = -14.53%, Сделок = 110
2024-01: PnL = -11.21%, Сделок = 130
2024-02: PnL = -12.69%, Сделок = 106
2024-03: PnL = -3.42%, Сделок = 118
2024-04: PnL = 16.65%, Сделок = 116
2024-05: PnL = -10.75%, Сделок = 115
2024-06: PnL = 3.50%, Сделок = 98
2024-07: PnL = 0.83%, Сделок = 117
2024-08: PnL = 0.37%, Сделок = 126
2024-09: PnL = 1.85%, Сделок = 123
2024-10: PnL = 5.68%, Сделок = 100
2024-11: PnL = -18.98%, Сделок = 120
1000PEPEUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End       

In [ ]:
manual_results_df.to_csv("manual_backtest_results.csv", index=False)

In [ ]:
window_length = 36
min_space = 5
sloping_atr_length = 23
take_profit_multiplier = 1.006
stop_loss_multiplier = 1.462


manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
)
# Если нужно сохранить результаты в CSV
manual_results_df = pd.DataFrame([manual_results])
manual_results_df
# manual_results_df.to_csv("manual_backtest_results.csv", index=False)


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

<ipython-input-28-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для ATOMUSDT (1h):
Общее количество сделок: 1731
Общий профит: 775.88%, Средняя длительность сделки: 4.49 ч.
Профитные месяцы: 31, Убыточные месяцы: 18
Средний профит в месяц: 50.28%, Средний убыток в месяц: -43.48%
Максимальный профит в месяц: 172.36%, Минимальный профит в месяц: -195.50%
Общий winrate: 62.16%, Winrate по месяцам: 63.27%

Анализ по месяцам:
2020-10: PnL = 6.58%, Сделок = 29
2020-11: PnL = 9.51%, Сделок = 29
2020-12: PnL = -3.61%, Сделок = 33
2021-01: PnL = -19.63%, Сделок = 38
2021-02: PnL = 18.76%, Сделок = 38
2021-03: PnL = 8.17%, Сделок = 33
2021-04: PnL = 56.50%, Сделок = 35
2021-05: PnL = 20.22%, Сделок = 39
2021-06: PnL = 54.67%, Сделок = 38
2021-07: PnL = 29.12%, Сделок = 33
2021-08: PnL = -3.30%, Сделок = 38
2021-09: PnL = 97.20%, Сделок = 36
2021-10: PnL = -15.91%, Сделок = 35
2021-11: PnL = -31.21%, Сделок = 35
2021-12: PnL = 84.50%, Сделок = 36
2022-01: PnL = 91.69%, Сделок = 29
2022-02: PnL = 79.68%, Сделок = 35
2022-03: PnL = 43.57%, 

In [ ]:
symbol = 'APEUSDT'
interval = '30m'
start_date = '2022-05'
end_date = '2024-10'
# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)

In [ ]:
window_length = 78
min_space =5
sloping_atr_length = 29
take_profit_multiplier = 1.48
stop_loss_multiplier = 2.59


manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
)
# Если нужно сохранить результаты в CSV
manual_results_df = pd.DataFrame([manual_results])
manual_results_df
# manual_results_df.to_csv("manual_backtest_results.csv", index=False)


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

<ipython-input-28-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для APEUSDT (30m):
Общее количество сделок: 1156
Общий профит: 190.72%, Средняя длительность сделки: 5.70 ч.
Профитные месяцы: 20, Убыточные месяцы: 10
Средний профит в месяц: 18.89%, Средний убыток в месяц: -18.71%
Максимальный профит в месяц: 43.54%, Минимальный профит в месяц: -34.80%
Общий winrate: 66.52%, Winrate по месяцам: 66.67%

Анализ по месяцам:
2022-05: PnL = 31.90%, Сделок = 30
2022-06: PnL = 1.26%, Сделок = 43
2022-07: PnL = 18.58%, Сделок = 35
2022-08: PnL = -13.99%, Сделок = 37
2022-09: PnL = 18.25%, Сделок = 42
2022-10: PnL = -8.26%, Сделок = 43
2022-11: PnL = 7.27%, Сделок = 31
2022-12: PnL = 1.23%, Сделок = 38
2023-01: PnL = 21.11%, Сделок = 43
2023-02: PnL = 19.84%, Сделок = 38
2023-03: PnL = 24.25%, Сделок = 41
2023-04: PnL = -1.03%, Сделок = 31
2023-05: PnL = -20.76%, Сделок = 37
2023-06: PnL = 40.98%, Сделок = 38
2023-07: PnL = -34.80%, Сделок = 39
2023-08: PnL = 11.53%, Сделок = 38
2023-09: PnL = -8.51%, Сделок = 32
2023-10: PnL = -31.57%, С

### Find best combinations

#### MAX Profit

In [ ]:
import pandas as pd

# Считаем данные из файла
file_path_by_month = '/content/drive/MyDrive/Sloping bot/hyperopt_optimization_results.csv'
df_optimized_by_month = pd.read_csv(file_path_by_month)
df_optimized_by_month

# Удалим дубликаты, если они есть
df_optimized_by_month.drop_duplicates(inplace=True)
df_optimized_by_month.reset_index(drop=True, inplace=True)

file_name = "hyperopt_optimization_results.csv"
df_optimized_by_month.to_csv(file_name, index=False)

df_optimized_by_month.info()


# Задаём пользовательский порядок сортировки для interval
interval_order = {'4h': 0, '1h': 1, '15m': 2, '5m': 3, '1m': 4}

# Добавляем новый столбец для пользовательской сортировки
df_filtered_by_month = df_optimized_by_month[df_optimized_by_month['profit_month_ratio%'] > 60].copy()
df_filtered_by_month['interval_order'] = df_filtered_by_month['interval'].map(interval_order)

# Сортировка по символу, interval_order и total_pnl% (по убыванию)
df_sorted_by_month = df_filtered_by_month.sort_values(
    by=['symbol', 'interval_order', 'total_pnl%'],
    ascending=[True, True, False]
).drop(columns=['interval_order'])  # Убираем временный столбец

# Сохраняем в CSV (если нужно)
df_sorted_by_month.to_csv('sorted_data_by_month.csv', index=False, encoding='utf-8')

print("Файл успешно сохранён как sorted_data_by_month.csv")
df_sorted_by_month

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47272 entries, 0 to 47271
Data columns (total 62 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   symbol                    47272 non-null  object 
 1   interval                  47272 non-null  object 
 2   window_length             47272 non-null  int64  
 3   min_space                 47272 non-null  int64  
 4   sloping_atr_length        47272 non-null  int64  
 5   take_profit_multiplier    47272 non-null  float64
 6   stop_loss_multiplier      47272 non-null  float64
 7   total_pnl%                47272 non-null  float64
 8   avg_trade_duration_hours  47272 non-null  float64
 9   profit_month_count        47272 non-null  int64  
 10  loss_month_count          47272 non-null  int64  
 11  2020-10                   38172 non-null  float64
 12  2020-11                   38172 non-null  float64
 13  2020-12                   38172 non-null  float64
 14  2021-0

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
21771,1000PEPEUSDT,1h,98,5,12,1.817388,1.809748,700.797272,10.322148,11,...,175.23,-12.53,163.23,-1.43,34.16,53.40,34.77,168.92,64.705882,NaN
18946,1000PEPEUSDT,1h,98,5,7,1.693179,2.360533,683.638414,12.476868,11,...,214.06,-85.78,188.20,-16.64,19.30,-14.33,18.94,184.99,64.705882,NaN
21736,1000PEPEUSDT,1h,98,5,7,1.661477,1.834137,629.728243,10.239203,12,...,172.45,-18.66,179.80,-23.53,22.95,22.06,25.40,153.57,70.588235,NaN
18891,1000PEPEUSDT,1h,92,5,7,1.457082,2.430666,621.659050,11.260000,14,...,178.86,-89.45,47.38,78.15,78.64,3.72,18.61,155.90,82.352941,NaN
21806,1000PEPEUSDT,1h,98,5,10,1.484407,2.238799,612.382192,9.550000,14,...,194.50,-66.93,89.04,18.54,18.71,22.76,26.43,160.70,82.352941,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14602,XRPUSDT,1h,24,5,17,2.029621,2.223821,7.201157,16.595761,30,...,11.40,2.92,-32.50,-16.39,-9.65,-13.19,3.02,4.82,61.224490,NaN
15237,XRPUSDT,1h,26,5,13,2.171290,1.633855,-2.098721,13.692363,31,...,12.22,7.48,-16.17,-1.30,0.06,3.04,8.99,9.52,63.265306,NaN
13686,XRPUSDT,1h,26,5,29,2.300613,2.459552,-12.219113,19.989218,30,...,18.54,4.18,-22.04,-7.02,-1.54,-17.15,11.59,1.32,61.224490,NaN
16577,XRPUSDT,1h,24,5,8,1.910206,1.957664,-19.014306,14.184783,30,...,6.64,14.10,-23.00,-4.72,-12.45,0.56,1.90,4.72,60.416667,NaN


In [ ]:
import pandas as pd
from itertools import combinations

# df = pd.read_excel("sloping final results.xlsx")
# df = pd.read_csv("sorted_data_by_month (3).csv")


df = df_sorted_by_month
# 2. Создание уникальных комбинаций "монета-интервал"
df["coin_interval"] = df["symbol"] + "_" + df["interval"]

# 3. Определение списка уникальных "монета-интервал"
coin_intervals = df["coin_interval"].unique()

# 4. Параметры анализа
combination_sizes = range(1, len(coin_intervals) + 1)  # Размеры комбинаций (от 1 до числа уникальных записей)
top_n = 10  # Количество лучших комбинаций для вывода

# 5. Список столбцов с ежемесячными профитами
monthly_columns = [col for col in df.columns if col.startswith("202")]

# 6. Перебор комбинаций
results = []
for size in combination_sizes:
    for combo in combinations(coin_intervals, size):
        # Проверка, что в комбинации каждая монета используется только один раз
        symbols = [item.split("_")[0] for item in combo]
        if len(symbols) != len(set(symbols)):  # Если есть дублирующиеся символы, пропускаем
            continue

        # Фильтрация данных по выбранным "монета-интервал"
        filtered_df = df[df["coin_interval"].isin(combo)]

        # Суммарный профит за год для комбинации
        total_pnl = filtered_df[monthly_columns].sum().sum()

        # Месячный профит (средний)
        avg_monthly_pnl = total_pnl / len(monthly_columns)

        # Минимальный месячный профит (убыток)
        min_monthly_pnl = filtered_df[monthly_columns].sum(axis=0).min()

        # PnL по месяцам
        monthly_pnl = filtered_df[monthly_columns].sum(axis=0).to_dict()

        # Сохранение результата
        results.append({
            "combo": combo,
            "total_pnl": total_pnl,
            "avg_monthly_pnl": avg_monthly_pnl,
            "min_monthly_pnl": min_monthly_pnl,
            "monthly_pnl": monthly_pnl,
        })

# 7. Сортировка комбинаций по общему профиту
sorted_results = sorted(results, key=lambda x: x["total_pnl"], reverse=True)

# 8. Вывод лучших комбинаций
print(f"Топ-{top_n} комбинаций:")
for i, result in enumerate(sorted_results[:top_n], 1):
    combo = ", ".join(result["combo"])
    print(f"{i}. Комбинация: {combo}")
    print(f"   Общий профит: {result['total_pnl']:.2f}%")
    print(f"   Средний месячный профит: {result['avg_monthly_pnl']:.2f}%")
    print(f"   Минимальный месячный профит (убыток): {result['min_monthly_pnl']:.2f}%")
    print(f"   PnL по месяцам:")
    for month, pnl in result["monthly_pnl"].items():
        print(f"      {month}: {pnl:.2f}%")



Топ-10 комбинаций:
1. Комбинация: ADAUSDT_1h, BTCUSDT_1h, FTMUSDT_1h, LINKUSDT_1h, XRPUSDT_1h
   Общий профит: 21175.27%
   Средний месячный профит: 1628.87%
   Минимальный месячный профит (убыток): -5202.81%
   PnL по месяцам:
      2023-10: -2422.86%
      2023-11: 3705.05%
      2023-12: 1436.60%
      2024-01: 693.46%
      2024-02: 287.22%
      2024-03: 14473.47%
      2024-04: -3199.81%
      2024-05: -5202.81%
      2024-06: 2518.86%
      2024-07: 3621.44%
      2024-08: -473.98%
      2024-09: 2961.99%
      2024-10: 2776.64%
2. Комбинация: ADAUSDT_1h, ATOMUSDT_1h, BTCUSDT_1h, FTMUSDT_1h, LINKUSDT_1h, XRPUSDT_1h
   Общий профит: 21126.79%
   Средний месячный профит: 1625.14%
   Минимальный месячный профит (убыток): -5235.45%
   PnL по месяцам:
      2023-10: -2547.12%
      2023-11: 3906.23%
      2023-12: 1894.63%
      2024-01: 872.83%
      2024-02: 208.59%
      2024-03: 14595.40%
      2024-04: -3342.15%
      2024-05: -5235.45%
      2024-06: 2590.48%
      2024-07: 367

#### MIN Risks

In [ ]:
import pandas as pd
from itertools import combinations

# Загрузка данных
file_path_by_month = '/content/drive/MyDrive/Sloping bot/hyperopt_optimization_results.csv'
df = pd.read_csv(file_path_by_month)

# Удаление дубликатов
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df = df[df['profit_month_count'] > 1].copy()

# Определяем столбцы с данными по месяцам
monthly_columns = [col for col in df.columns if col.startswith("202")]

# Фильтрация: топ-10 по `total_pnl%` и `profit_month_ratio%` для каждой монеты
# Топ-10 по `total_pnl%`
top_total_pnl = (
    df.sort_values(by="total_pnl%", ascending=False)
    .groupby("symbol")
    .head(10)
)

# Топ-10 по `profit_month_ratio%`
top_month_ratio = (
    df.sort_values(by="profit_month_ratio%", ascending=False)
    .groupby("symbol")
    .head(10)
)

# Объединение и удаление дубликатов
filtered_top = pd.concat([top_total_pnl, top_month_ratio]).drop_duplicates()

# Финальная сортировка для удобства
filtered_top = filtered_top.sort_values(
    by=["symbol", "interval", "total_pnl%"],
    ascending=[True, True, False]
)

filtered_top = filtered_top[(filtered_top['profit_month_ratio%'] > 70) & (filtered_top['min_space'] == 5)].copy()
# Сохранение предварительно отфильтрованных данных
filtered_top.to_csv("filtered_top_models.csv", index=False)
print("Файл успешно сохранён как filtered_top_models.csv")
filtered_top

Файл успешно сохранён как filtered_top_models.csv


,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
21736,1000PEPEUSDT,1h,98,5,7,1.661477,1.834137,629.728243,10.239203,12,...,172.45,-18.66,179.80,-23.53,22.95,22.06,25.40,153.57,70.588235,NaN
18891,1000PEPEUSDT,1h,92,5,7,1.457082,2.430666,621.659050,11.260000,14,...,178.86,-89.45,47.38,78.15,78.64,3.72,18.61,155.90,82.352941,NaN
21806,1000PEPEUSDT,1h,98,5,10,1.484407,2.238799,612.382192,9.550000,14,...,194.50,-66.93,89.04,18.54,18.71,22.76,26.43,160.70,82.352941,NaN
19285,1000PEPEUSDT,1h,94,5,11,1.481311,2.416515,510.363391,10.325342,14,...,117.17,-25.59,39.09,92.14,45.53,15.26,11.88,103.98,82.352941,NaN
19274,1000PEPEUSDT,1h,96,5,11,1.482654,2.223942,473.693979,9.178218,14,...,129.53,-35.89,30.38,6.43,24.20,34.73,18.39,155.59,82.352941,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15101,XRPUSDT,1h,42,5,17,1.758816,1.854125,857.496764,11.730132,36,...,161.11,-169.84,-111.03,-40.49,47.25,10.10,37.27,15.07,73.469388,NaN
15883,XRPUSDT,1h,42,5,16,2.577894,2.417126,773.604304,24.419664,36,...,202.23,-33.25,-113.22,32.64,41.90,-56.35,85.87,64.58,73.469388,NaN
14582,XRPUSDT,1h,42,5,15,1.551276,2.561768,771.772117,13.895495,35,...,208.83,-29.31,-104.29,25.71,115.66,27.84,-26.18,4.81,71.428571,NaN
11262,XRPUSDT,1h,42,5,9,1.807181,1.921478,567.091708,12.918033,37,...,138.83,-117.53,-65.91,-37.51,13.67,25.34,24.72,35.32,75.510204,NaN


In [ ]:
# скачиваем все модели с диска

# import pandas as pd

# # Считаем данные из файла
# file_path_by_month = '/content/drive/MyDrive/Sloping bot/hyperopt_optimization_results.csv'
# df_optimized_by_month = pd.read_csv(file_path_by_month)
# df_optimized_by_month

# # Удалим дубликаты, если они есть
# df_optimized_by_month.drop_duplicates(inplace=True)
# df_optimized_by_month.reset_index(drop=True, inplace=True)

# file_name = "hyperopt_optimization_results.csv"
# df_optimized_by_month.to_csv(file_name, index=False)

# df_optimized_by_month.info()


# # Задаём пользовательский порядок сортировки для interval
# interval_order = {'4h': 0, '1h': 1, '15m': 2, '5m': 3, '1m': 4}

# # Добавляем новый столбец для пользовательской сортировки
# df_filtered_by_month = df_optimized_by_month[df_optimized_by_month['profit_month_ratio%'] > 70].copy()
# df_filtered_by_month['interval_order'] = df_filtered_by_month['interval'].map(interval_order)

# # Сортировка по символу, interval_order и total_pnl% (по убыванию)
# df_sorted_by_month = df_filtered_by_month.sort_values(
#     by=['symbol', 'interval_order', 'total_pnl%'],
#     ascending=[True, True, False]
# ).drop(columns=['interval_order'])  # Убираем временный столбец

# # Сохраняем в CSV (если нужно)
# df_sorted_by_month.to_csv('sorted_data_by_month.csv', index=False, encoding='utf-8')

# print("Файл успешно сохранён как sorted_data_by_month.csv")
# df_sorted_by_month

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50172 entries, 0 to 50171
Data columns (total 62 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   symbol                    50172 non-null  object 
 1   interval                  50172 non-null  object 
 2   window_length             50172 non-null  int64  
 3   min_space                 50172 non-null  int64  
 4   sloping_atr_length        50172 non-null  int64  
 5   take_profit_multiplier    50172 non-null  float64
 6   stop_loss_multiplier      50172 non-null  float64
 7   total_pnl%                50172 non-null  float64
 8   avg_trade_duration_hours  50172 non-null  float64
 9   profit_month_count        50172 non-null  int64  
 10  loss_month_count          50172 non-null  int64  
 11  2020-10                   39722 non-null  float64
 12  2020-11                   39722 non-null  float64
 13  2020-12                   39722 non-null  float64
 14  2021-0

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
21736,1000PEPEUSDT,1h,98,5,7,1.661477,1.834137,629.728243,10.239203,12,...,172.45,-18.66,179.80,-23.53,22.95,22.06,25.40,153.57,70.588235,NaN
18891,1000PEPEUSDT,1h,92,5,7,1.457082,2.430666,621.659050,11.260000,14,...,178.86,-89.45,47.38,78.15,78.64,3.72,18.61,155.90,82.352941,NaN
21806,1000PEPEUSDT,1h,98,5,10,1.484407,2.238799,612.382192,9.550000,14,...,194.50,-66.93,89.04,18.54,18.71,22.76,26.43,160.70,82.352941,NaN
19264,1000PEPEUSDT,1h,92,5,8,1.089490,2.507078,612.128629,7.924699,14,...,139.73,-36.66,9.18,59.87,51.82,49.91,45.29,164.63,82.352941,NaN
18704,1000PEPEUSDT,1h,72,5,10,1.418003,2.246889,606.723701,9.440111,12,...,120.03,-57.45,6.45,134.64,143.95,35.89,34.10,98.77,70.588235,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15101,XRPUSDT,1h,42,5,17,1.758816,1.854125,857.496764,11.730132,36,...,161.11,-169.84,-111.03,-40.49,47.25,10.10,37.27,15.07,73.469388,NaN
15883,XRPUSDT,1h,42,5,16,2.577894,2.417126,773.604304,24.419664,36,...,202.23,-33.25,-113.22,32.64,41.90,-56.35,85.87,64.58,73.469388,NaN
14582,XRPUSDT,1h,42,5,15,1.551276,2.561768,771.772117,13.895495,35,...,208.83,-29.31,-104.29,25.71,115.66,27.84,-26.18,4.81,71.428571,NaN
11262,XRPUSDT,1h,42,5,9,1.807181,1.921478,567.091708,12.918033,37,...,138.83,-117.53,-65.91,-37.51,13.67,25.34,24.72,35.32,75.510204,NaN


In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations, product
from tqdm import tqdm
from google.colab import drive
import os
import shutil


# filtered_top = pd.read_excel("final models.xlsx")


# Подключение Google Drive
drive.mount('/content/drive')
shared_folder_path = '/content/drive/MyDrive/Sloping bot'

# Локальный путь для сохранения
local_file_name = "final_combos.csv"
drive_file_name = "final_combos.csv"

# Загрузка данных
# df = pd.read_csv("filtered_top_models (4).csv")
# df = pd.read_excel("df.xlsx")
# df = filtered_top

df = pd.read_excel("final models.xlsx")

# Параметры ввода
start_date = "2023-10"
end_date = "2024-11"

# Параметры фильтрации
max_loss_threshold = -10  # Заданный порог убытка
max_symbols = 7          # Максимальное количество монет
results = []



# Генерация списка месячных колонок
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')  # MS = начало месяца
monthly_columns = [date.strftime("%Y-%m") for date in date_range]

# Проверка на пропуски и замена NaN
df[monthly_columns] = df[monthly_columns].replace("", np.nan).astype(float)

# Обработка всех комбинаций
for size in range(1, min(max_symbols, len(df['symbol'].unique())) + 1):
    for symbol_combo in combinations(df['symbol'].unique(), size):
        models_combo = [df[df['symbol'] == symbol].to_dict(orient="records") for symbol in symbol_combo]
        parameter_combinations = product(*models_combo)

        # Убираем лишние сообщения tqdm
        for combo in tqdm(parameter_combinations, desc=f"Processing {size}-coin combinations", leave=False):
            total_pnl = sum(item['total_pnl%'] for item in combo)
            avg_monthly_pnl = total_pnl / len(monthly_columns)
            monthly_pnl = {col: sum(item[col] for item in combo if not pd.isna(item[col])) for col in monthly_columns}
            min_monthly_pnl = min(monthly_pnl.values(), default=0)

            # Условия фильтрации
            if min_monthly_pnl < max_loss_threshold:
                continue

            # Форматирование параметров моделей
            models_params = "; ".join([
                f"{item['symbol']}_{item['interval']} "
                f"(window={item['window_length']}, atr_length={item['sloping_atr_length']}, "
                f"min_space={item['min_space']}, take={item['take_profit_multiplier']}, loss={item['stop_loss_multiplier']})"
                for item in combo
            ])

            results.append({
                "combo": ", ".join(symbol_combo),
                "total_pnl": total_pnl,
                "avg_monthly_pnl": avg_monthly_pnl,
                "min_monthly_pnl": min_monthly_pnl,
                **monthly_pnl,
                "models_params": models_params
            })

# Сортировка результатов по общему профиту
results_df = pd.DataFrame(results)
top_results = results_df.sort_values(by="total_pnl", ascending=False).head(10)

# Сохранение с вашей функцией
if not top_results.empty:
    save_results_to_csv(top_results, shared_folder_path, drive_file_name)  # На Google Drive
    save_results_to_csv(top_results, ".", local_file_name)  # Локально
else:
    print("Не найдено подходящих комбинаций!")

# Вывод результатов в консоль
if not top_results.empty:
    for i, result in enumerate(top_results.to_dict(orient="records"), start=1):
        print('---' * 25)
        print(f"{i}. Комбинация: {result['combo']}")
        print(f"   Общий профит: {result['total_pnl']:.2f}%")
        print(f"   Средний месячный профит: {result['avg_monthly_pnl']:.2f}%")
        print(f"   Минимальный месячный профит: {result['min_monthly_pnl']:.2f}%")
        print(f"   PnL по месяцам:")
        for month in monthly_columns:
            if month in result:
                print(f"      {month}: {result[month]:.2f}%")
        print(f"   Параметры моделей: {result['models_params']}")
        print()  # Пустая строка между комбинациями
else:
    print("Не найдено подходящих комбинаций!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Резервная копия создана: /content/drive/MyDrive/Sloping bot/final_combos.csv.backup
Обновленные данные сохранены в /content/drive/MyDrive/Sloping bot/final_combos.csv
Файл не существует, будет создан новый файл: ./final_combos.csv
Обновленные данные сохранены в ./final_combos.csv
---------------------------------------------------------------------------
1. Комбинация: 1000PEPEUSDT, APEUSDT, LINKUSDT, ORDIUSDT, SOLUSDT, WIFUSDT, WLDUSDT
   Общий профит: 2570.76%
   Средний месячный профит: 183.63%
   Минимальный месячный профит: 18.80%
   PnL по месяцам:
      2023-10: 18.80%
      2023-11: 220.52%
      2023-12: 26.78%
      2024-01: 78.75%
      2024-02: 73.96%
      2024-03: 159.58%
      2024-04: 180.41%
      2024-05: 151.92%
      2024-06: 247.32%
      2024-07: 306.81%
      2024-08: 281.30%
      2024-09: 115.11%
      2024-10: 477.22%
      2024-11: 288.68%
   Параметры моделей: 1000PEPEUSDT_30m (window=86, atr_length=18, min_space=5, take=1.80623414832759, loss=2.599026140883

In [ ]:
# import numpy as np
# import pandas as pd
# from itertools import combinations, product
# from tqdm import tqdm
# from pandas.tseries.offsets import MonthBegin

# # Загрузка данных
# df = pd.read_excel("df.xlsx")
# # df = pd.read_csv("filtered_top_models (4).csv")
# # Параметры ввода
# start_date = "2023-10"
# end_date = "2024-11"

# # Генерация списка месячных колонок
# date_range = pd.date_range(start=start_date, end=end_date, freq='MS')  # MS = начало месяца
# monthly_columns = [date.strftime("%Y-%m") for date in date_range]

# # Проверка на пропуски и замена NaN
# df[monthly_columns] = df[monthly_columns].replace("", np.nan).astype(float)

# # Параметры фильтрации
# max_loss_threshold = -10  # Заданный порог убытка
# max_symbols = 3           # Максимальное количество монет
# results = []

# # Проходим по всем возможным комбинациям до 5 символов
# for size in range(1, min(max_symbols, len(df['symbol'].unique())) + 1):
#     for symbol_combo in combinations(df['symbol'].unique(), size):
#         # print(f"Обрабатываем комбинацию монет: {symbol_combo}")

#         models_combo = [df[df['symbol'] == symbol].to_dict(orient="records") for symbol in symbol_combo]
#         parameter_combinations = product(*models_combo)

#         for combo in tqdm(parameter_combinations): #desc="Processing combinations"
#             total_pnl = sum(item['total_pnl%'] for item in combo)
#             avg_monthly_pnl = total_pnl / len(monthly_columns)
#             monthly_pnl = {col: sum(item[col] for item in combo if not pd.isna(item[col])) for col in monthly_columns}
#             min_monthly_pnl = min(monthly_pnl.values(), default=0)

#             # Условия фильтрации
#             if min_monthly_pnl < max_loss_threshold:
#                 continue

#             results.append({
#                 "combo": symbol_combo,
#                 "total_pnl": total_pnl,
#                 "avg_monthly_pnl": avg_monthly_pnl,
#                 "min_monthly_pnl": min_monthly_pnl,
#                 "monthly_pnl": monthly_pnl,
#                 "models_params": [{**item, "coin_interval": f"{item['symbol']}_{item['interval']}"} for item in combo]
#             })

# # Сортировка результатов по общему профиту и выбор топ-10
# top_results = sorted(results, key=lambda x: x["total_pnl"], reverse=True)[:10]

# # Форматированный вывод результатов
# if not top_results:
#     print("Не найдено подходящих комбинаций!")
# else:
#     for i, result in enumerate(top_results, start=1):
#         print(f"{i}. Комбинация: {', '.join(['{}_{}'.format(symbol, item['interval']) for symbol, item in zip(result['combo'], result['models_params'])])}")
#         print(f"   Общий профит: {result['total_pnl']:.2f}%")
#         print(f"   Средний месячный профит: {result['avg_monthly_pnl']:.2f}%")
#         print(f"   Минимальный месячный профит: {result['min_monthly_pnl']:.2f}%")
#         print(f"   PnL по месяцам:")
#         for month, pnl in result['monthly_pnl'].items():
#             print(f"      {month}: {pnl:.2f}%")
#         print(f"   Параметры моделей в комбинации:")
#         for params in result['models_params']:
#             print(f"      {params}")
#         print()  # Пустая строка между комбинациями

In [ ]:
# import numpy as np
# import pandas as pd
# from itertools import combinations, product
# from tqdm import tqdm
# from pandas.tseries.offsets import MonthBegin

# # Загрузка данных
# df = pd.read_excel("df.xlsx")
# # df = filtered_top
# # Параметры ввода
# start_date = "2023-10"
# end_date = "2024-11"

# # Генерация списка месячных колонок
# date_range = pd.date_range(start=start_date, end=end_date, freq='MS')  # MS = начало месяца
# monthly_columns = [date.strftime("%Y-%m") for date in date_range]

# # Проверка на пропуски и замена NaN
# df[monthly_columns] = df[monthly_columns].replace("", np.nan).astype(float)

# # Параметры фильтрации
# max_loss_threshold = -10 # Заданный порог убытка
# results = []

# # Проходим по всем возможным комбинациям
# for size in range(1, len(df['symbol'].unique()) + 1):
#     for symbol_combo in combinations(df['symbol'].unique(), size):
#         print(f"Обрабатываем комбинацию монет: {symbol_combo}")

#         models_combo = [df[df['symbol'] == symbol].to_dict(orient="records") for symbol in symbol_combo]
#         parameter_combinations = product(*models_combo)

#         for combo in tqdm(parameter_combinations, desc="Processing combinations"):
#             total_pnl = sum(item['total_pnl%'] for item in combo)
#             avg_monthly_pnl = total_pnl / len(monthly_columns)
#             monthly_pnl = {col: sum(item[col] for item in combo if not pd.isna(item[col])) for col in monthly_columns}
#             min_monthly_pnl = min(monthly_pnl.values(), default=0)

#             # Условия фильтрации
#             if min_monthly_pnl < max_loss_threshold:
#                 continue

#             results.append({
#                 "combo": symbol_combo,
#                 "total_pnl": total_pnl,
#                 "avg_monthly_pnl": avg_monthly_pnl,
#                 "min_monthly_pnl": min_monthly_pnl,
#                 "monthly_pnl": monthly_pnl,
#                 "models_params": [{**item, "coin_interval": f"{item['symbol']}_{item['interval']}"} for item in combo]
#             })

# # Сортировка результатов по общему профиту и выбор топ-10
# top_results = sorted(results, key=lambda x: x["total_pnl"], reverse=True)[:10]

# # Форматированный вывод результатов
# if not top_results:
#     print("Не найдено подходящих комбинаций!")
# else:
#     for i, result in enumerate(top_results, start=1):
#         print(f"{i}. Комбинация: {', '.join(['{}_{}'.format(symbol, item['interval']) for symbol, item in zip(result['combo'], result['models_params'])])}")
#         print(f"   Общий профит: {result['total_pnl']:.2f}%")
#         print(f"   Средний месячный профит: {result['avg_monthly_pnl']:.2f}%")
#         print(f"   Минимальный месячный профит: {result['min_monthly_pnl']:.2f}%")
#         print(f"   PnL по месяцам:")
#         for month, pnl in result['monthly_pnl'].items():
#             print(f"      {month}: {pnl:.2f}%")
#         print(f"   Параметры моделей в комбинации:")
#         for params in result['models_params']:
#             print(f"      {params}")
#         print()  # Пустая строка между комбинациями




#### final Backtest

###### Тест 8/12/2024

In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest, Strategy


# Класс стратегии для backtesting
class SlopingStrategy(Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5
    take_profit_multiplier = 1.5
    trade_on_close = True
    leverage = 1.0

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])
        signal = self.slope_indicator.get_value()

        if signal and signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass

        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
                self.sell(sl=stop_loss, tp=take_profit)
            except:
                pass


# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    return df


def analyze_and_display_results(trades_combined, initial_cash):
    # Преобразуем список сделок в DataFrame
    df = pd.DataFrame(trades_combined)

    # Сортировка сделок по времени открытия
    df['OpenDate'] = pd.to_datetime(df['OpenDate'])
    df['CloseDate'] = pd.to_datetime(df['CloseDate'])
    df.sort_values(by='OpenDate', inplace=True)

    # Вычисление времени длительности сделок в часах
    df['Duration'] = (df['CloseDate'] - df['OpenDate']).dt.total_seconds() / 3600  # В часах

    # Инициализация баланса
    balance = initial_cash
    balances = []
    cumulative_pnls = []

    # Последовательный пересчёт PnL и баланса
    for _, trade in df.iterrows():
        balance += trade['PnL']
        cumulative_pnls.append(balance - initial_cash)
        balances.append(balance)

    # Добавление пересчитанных данных в DataFrame
    df['CumulativePnL'] = cumulative_pnls
    df['Balance'] = balances
    df['CumulativePnLPercent'] = (df['CumulativePnL'] / initial_cash) * 100

    # Сохранение детальной информации в файл
    df.to_csv('sequential_backtest_results.csv', index=False)
    print("Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.")

    # Группировка по месяцам и анализ
    monthly_stats = []
    grouped = df.groupby(df['OpenDate'].dt.to_period('M'))
    for month, group in grouped:
        total_pnl = group['PnL'].sum()
        percent_pnl = (total_pnl / initial_cash) * 100
        ending_balance = group['Balance'].iloc[-1]
        total_trades = len(group)

        # Среднее время длительности сделок в часах
        avg_duration = group['Duration'].mean()

        # Разделение сделок на профитные и убыточные
        profitable_trades = group[group['PnL'] > 0]
        losing_trades = group[group['PnL'] <= 0]

        # Разделение по монетам
        trades_by_symbol = group.groupby('Symbol')['PnL'].agg(['count', 'sum']).reset_index()
        symbol_stats = {
            row.Symbol: {
                'trades': row['count'],
                'pnl': row['sum'],
                'percent_pnl': (row['sum'] / initial_cash) * 100
            }
            for _, row in trades_by_symbol.iterrows()
        }

        # Подсчет количества сделок по каждому символу
        profitable_counts = profitable_trades['Symbol'].value_counts()
        losing_counts = losing_trades['Symbol'].value_counts()

        monthly_stats.append({
            'month': str(month),
            'total_pnl': total_pnl,
            'percent_pnl': percent_pnl,
            'ending_balance': ending_balance,
            'total_trades': total_trades,
            'avg_duration_hours': avg_duration,  # Средняя длительность сделок в часах
            'profitable_trades': len(profitable_trades),
            'losing_trades': len(losing_trades),
            'trades_by_symbol': symbol_stats,
            'profitable_by_symbol': profitable_counts.to_dict(),
            'losing_by_symbol': losing_counts.to_dict()
        })

    # Вывод статистики по месяцам
    print("\n===== Статистика по месяцам =====\n")
    for stat in monthly_stats:
        print(f"Месяц: {stat['month']}")
        print(f"Общий PnL: {stat['total_pnl']:.2f} ({stat['percent_pnl']:.2f}%)")
        print(f"Баланс на конец месяца: {stat['ending_balance']:.2f}")
        print(f"Количество сделок: {stat['total_trades']} (разбивка по монетам: {stat['trades_by_symbol']})")
        print(f"Средняя длительность сделок: {stat['avg_duration_hours']:.2f} часов")
        print(f"Профитных сделок: {stat['profitable_trades']} (по монетам: {stat['profitable_by_symbol']})")
        print(f"Убыточных сделок: {stat['losing_trades']} (по монетам: {stat['losing_by_symbol']})\n")
    print("=================================\n")

    # Сохранение статистики по месяцам в CSV
    stats_df = pd.DataFrame(monthly_stats)
    stats_df.to_csv('monthly_stats.csv', index=False)
    print("Статистика по месяцам сохранена в 'monthly_stats.csv'.")


def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    all_trades = []  # Список для хранения всех сделок

    for coin in coins_config:
        print(f"Запуск тестирования для {coin['symbol']}...")

        # Загрузка данных
        data = download_klines(coin['symbol'], coin['interval'], start_date, end_date)

        # Настройка параметров стратегии
        strategy_config = {
            'window_length': coin['window_length'],
            'min_space': coin['min_space'],
            'sloping_atr_length': coin['sloping_atr_length'],
            'take_profit_multiplier': coin['take_profit_multiplier'],
            'stop_loss_multiplier': coin['stop_loss_multiplier'],
            'leverage': coin['leverage']
        }

        # Запуск бэктеста
        bt = Backtest(
            data,
            SlopingStrategy,
            cash=initial_cash,
            trade_on_close=True,
            exclusive_orders=True,
            commission=0.0005
        )
        stats = bt.run(**strategy_config)

        # Извлечение данных о сделках
        trades = stats['_trades']
        if not trades.empty:
            for _, trade in trades.iterrows():
                all_trades.append({
                    'Symbol': coin['symbol'],
                    'OpenDate': trade['EntryTime'],
                    'CloseDate': trade['ExitTime'],
                    'PnL': trade['PnL'],
                    'EntryPrice': trade['EntryPrice'],
                    'ExitPrice': trade['ExitPrice'],
                    'Size': trade['Size']
                })

        print(f"Тестирование для {coin['symbol']} завершено.")

    return all_trades


# Основная функция запуска
def run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash):
    trades_combined = run_combined_backtest(coins_config, start_date, end_date, initial_cash)
    if trades_combined:
        analyze_and_display_results(trades_combined, initial_cash)
    else:
        print("Сделок не найдено. Проверьте данные и параметры стратегии.")

In [ ]:
# final_models_check = pd.read_excel("final_models_check.xlsx")

In [ ]:
import pandas as pd
import re

def parse_final_models_check(file_path):
    coins_config = []

    # Регулярное выражение для извлечения параметров монеты из строки
    model_params_pattern = re.compile(
        r"(?P<symbol>\w+)_((?P<interval>\d+[mh])) \(window=(?P<window>\d+), atr_length=(?P<atr_length>\d+), min_space=(?P<min_space>\d+), take=(?P<take>[\d.]+), loss=(?P<loss>[\d.]+)"
    )

    # Чтение файла Excel в DataFrame
    data = pd.read_excel(file_path)

    for _, row in data.iterrows():
        models_params = row["models_params"]

        for match in model_params_pattern.finditer(models_params):
            symbol = match.group("symbol")
            interval = match.group("interval")
            coins_config.append({
                "symbol": symbol,
                "interval": interval,
                "window_length": int(match.group("window")),
                "min_space": int(match.group("min_space")),
                "sloping_atr_length": int(match.group("atr_length")),
                "take_profit_multiplier": float(match.group("take")),
                "stop_loss_multiplier": float(match.group("loss")),
                "trade_amount": 10,  # Укажите ваше значение
                "leverage": 20  # Укажите ваше значение
            })

    return coins_config

# Пример использования для Google Colab
file_path = "final_models_check 1.xlsx"  # Укажите путь к вашему файлу Excel
coins_config = parse_final_models_check(file_path)

# Для проверки результата
for coin in coins_config:
    print(coin)


{'symbol': '1000PEPEUSDT', 'interval': '30m', 'window_length': 86, 'min_space': 5, 'sloping_atr_length': 18, 'take_profit_multiplier': 1.80623414832759, 'stop_loss_multiplier': 2.59902614088353, 'trade_amount': 10, 'leverage': 20}
{'symbol': 'APEUSDT', 'interval': '1h', 'window_length': 90, 'min_space': 5, 'sloping_atr_length': 20, 'take_profit_multiplier': 2.01297570047132, 'stop_loss_multiplier': 2.3111568893801, 'trade_amount': 10, 'leverage': 20}
{'symbol': 'ORDIUSDT', 'interval': '1h', 'window_length': 40, 'min_space': 5, 'sloping_atr_length': 30, 'take_profit_multiplier': 2.46269916422318, 'stop_loss_multiplier': 1.00722624957696, 'trade_amount': 10, 'leverage': 20}
{'symbol': 'SOLUSDT', 'interval': '1h', 'window_length': 100, 'min_space': 5, 'sloping_atr_length': 16, 'take_profit_multiplier': 2.030368420358, 'stop_loss_multiplier': 1.8828792771351, 'trade_amount': 10, 'leverage': 20}
{'symbol': 'SUIUSDT', 'interval': '1h', 'window_length': 10, 'min_space': 5, 'sloping_atr_length

Комбинация: 1000PEPEUSDT, BNBUSDT, LINAUSDT, ORDIUSDT, WIFUSDT, WLDUSDT, XRPUSDT
   Общий профит: 6020.08%
   Средний месячный профит: 430.01%
   Минимальный месячный профит: -2.10%
   PnL по месяцам:
      2023-10: 162.65%
      2023-11: 236.37%
      2023-12: -2.10%
      2024-01: 223.20%
      2024-02: 239.31%
      2024-03: 192.12%
      2024-04: 33.95%
      2024-05: 27.71%
      2024-06: 58.65%
      2024-07: 298.94%
      2024-08: 251.16%
      2024-09: 96.04%
      2024-10: 287.84%
      2024-11: 311.65%
   Параметры моделей: 1000PEPEUSDT_1h (window=100, atr_length=9, min_space=5, take=1.8740350422203365, loss=2.09759483903459); BNBUSDT_1h (window=34, atr_length=20, min_space=5, take=1.0003717941136006, loss=2.39353768020516); LINAUSDT_1h (window=96, atr_length=27, min_space=5, take=2.049448110407128, loss=2.0580648628801086); ORDIUSDT_1h (window=42, atr_length=30, min_space=5, take=2.561307156240609, loss=1.0179889676917246); WIFUSDT_1h (window=84, atr_length=7, min_space=5, take=2.003858833178518, loss=2.3952446942683343); WLDUSDT_1h (window=68, atr_length=29, min_space=5, take=2.42160013881293, loss=2.551619203185885); XRPUSDT_1h (window=42, atr_length=17, min_space=5, take=1.7588158439140311, loss=1.854125401775483)


In [ ]:
start_date = '2024-11'
end_date = '2024-11'
initial_cash = 30
display(coins_config)
# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

[{'symbol': '1000PEPEUSDT',
  'interval': '30m',
  'window_length': 86,
  'min_space': 5,
  'sloping_atr_length': 18,
  'take_profit_multiplier': 1.80623414832759,
  'stop_loss_multiplier': 2.59902614088353,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'APEUSDT',
  'interval': '1h',
  'window_length': 90,
  'min_space': 5,
  'sloping_atr_length': 20,
  'take_profit_multiplier': 2.01297570047132,
  'stop_loss_multiplier': 2.3111568893801,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ORDIUSDT',
  'interval': '1h',
  'window_length': 40,
  'min_space': 5,
  'sloping_atr_length': 30,
  'take_profit_multiplier': 2.46269916422318,
  'stop_loss_multiplier': 1.00722624957696,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'SOLUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 16,
  'take_profit_multiplier': 2.030368420358,
  'stop_loss_multiplier': 1.8828792771351,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'SUIUSDT',
 

Запуск тестирования для 1000PEPEUSDT...
Тестирование для 1000PEPEUSDT завершено.
Запуск тестирования для APEUSDT...
Тестирование для APEUSDT завершено.
Запуск тестирования для ORDIUSDT...


<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для ORDIUSDT завершено.
Запуск тестирования для SOLUSDT...


<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Тестирование для SUIUSDT завершено.
Запуск тестирования для WIFUSDT...
Тестирование для WIFUSDT завершено.
Запуск тестирования для WLDUSDT...
Тестирование для WLDUSDT завершено.
Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.

===== Статистика по месяцам =====

Месяц: 2024-11
Общий PnL: 19.15 (63.83%)
Баланс на конец месяца: 49.15
Количество сделок: 99 (разбивка по монетам: {'1000PEPEUSDT': {'trades': 29, 'pnl': -1.8386401830665606, 'percent_pnl': -6.1288006102218695}, 'APEUSDT': {'trades': 15, 'pnl': 4.428885235759581, 'percent_pnl': 14.76295078586527}, 'SUIUSDT': {'trades': 22, 'pnl': 7.046264616889257, 'percent_pnl': 23.48754872296419}, 'WIFUSDT': {'trades': 21, 'pnl': -0.18736708857191697, 'percent_pnl': -0.62455696190639}, 'WLDUSDT': {'trades': 12, 'pnl': 9.699551178393488, 'percent_pnl': 32.33183726131163}})
Средняя длительность сделок: 13.91 часов
Профитных сд

<ipython-input-31-a9b4d2fccb8d>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))


In [ ]:
start_date = '2020-10'
end_date = '2024-11'
initial_cash = 30
display(coins_config)
# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

[{'symbol': '1000PEPEUSDT',
  'interval': '30m',
  'window_length': 86,
  'min_space': 5,
  'sloping_atr_length': 18,
  'take_profit_multiplier': 1.80623414832759,
  'stop_loss_multiplier': 2.59902614088353,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ADAUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 14,
  'take_profit_multiplier': 1.930628726507,
  'stop_loss_multiplier': 2.44128263607204,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'APEUSDT',
  'interval': '1h',
  'window_length': 90,
  'min_space': 5,
  'sloping_atr_length': 20,
  'take_profit_multiplier': 2.01297570047132,
  'stop_loss_multiplier': 2.3111568893801,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ARBUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 14,
  'take_profit_multiplier': 2.00967846755187,
  'stop_loss_multiplier': 2.09159862679683,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'DOGEUSDT',

Запуск тестирования для 1000PEPEUSDT...
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-202

<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для ETHUSDT завершено.
Запуск тестирования для LINAUSDT...
Ошибка: Файл klines/LINAUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Тестирование для LINAUSDT завершено.
Запуск тестирования для LINKUSDT...


<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


...

Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Ошибка: Файл klines/SUIUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-08.zip поврежден или не является 

<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для TRBUSDT завершено.
Запуск тестирования для WIFUSDT...
Ошибка: Файл klines/WIFUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-08.zip поврежден или не является 

<ipython-input-31-a9b4d2fccb8d>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))



===== Статистика по месяцам =====

Месяц: 2020-10
Общий PnL: 28.21 (94.03%)
Баланс на конец месяца: 58.21
Количество сделок: 98 (разбивка по монетам: {'ADAUSDT': {'trades': 16, 'pnl': 5.50422192830471, 'percent_pnl': 18.347406427682365}, 'DOGEUSDT': {'trades': 15, 'pnl': -1.2412584086804594, 'percent_pnl': -4.137528028934865}, 'EOSUSDT': {'trades': 18, 'pnl': 1.6663959464138434, 'percent_pnl': 5.554653154712811}, 'LINKUSDT': {'trades': 21, 'pnl': 2.0794031186057165, 'percent_pnl': 6.931343728685721}, 'SOLUSDT': {'trades': 15, 'pnl': 10.681883951219923, 'percent_pnl': 35.60627983739975}, 'TRBUSDT': {'trades': 13, 'pnl': 9.517557316126414, 'percent_pnl': 31.725191053754713}})
Средняя длительность сделок: 13.04 часов
Профитных сделок: 68 (по монетам: {'LINKUSDT': 15, 'EOSUSDT': 13, 'ADAUSDT': 13, 'SOLUSDT': 11, 'TRBUSDT': 9, 'DOGEUSDT': 7})
Убыточных сделок: 30 (по монетам: {'DOGEUSDT': 8, 'LINKUSDT': 6, 'EOSUSDT': 5, 'SOLUSDT': 4, 'TRBUSDT': 4, 'ADAUSDT': 3})

Месяц: 2020-11
Общий PnL: 

In [ ]:
start_date = '2020-10'
end_date = '2024-11'
initial_cash = 30
display(coins_config)
# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

[{'symbol': '1000PEPEUSDT',
  'interval': '30m',
  'window_length': 86,
  'min_space': 5,
  'sloping_atr_length': 18,
  'take_profit_multiplier': 1.80623414832759,
  'stop_loss_multiplier': 2.59902614088353,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ADAUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 14,
  'take_profit_multiplier': 1.930628726507,
  'stop_loss_multiplier': 2.44128263607204,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'APEUSDT',
  'interval': '1h',
  'window_length': 90,
  'min_space': 5,
  'sloping_atr_length': 20,
  'take_profit_multiplier': 2.01297570047132,
  'stop_loss_multiplier': 2.3111568893801,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ARBUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 14,
  'take_profit_multiplier': 2.00967846755187,
  'stop_loss_multiplier': 2.09159862679683,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'DOGEUSDT',

Запуск тестирования для 1000PEPEUSDT...
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-202

<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для ETHUSDT завершено.
Запуск тестирования для LINAUSDT...
Ошибка: Файл klines/LINAUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Тестирование для LINAUSDT завершено.
Запуск тестирования для LINKUSDT...


<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


...

Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Ошибка: Файл klines/SUIUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-08.zip поврежден или не является 

<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для TRBUSDT завершено.
Запуск тестирования для WIFUSDT...
Ошибка: Файл klines/WIFUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-08.zip поврежден или не является 

<ipython-input-31-a9b4d2fccb8d>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))



===== Статистика по месяцам =====

Месяц: 2020-10
Общий PnL: 26.54 (88.47%)
Баланс на конец месяца: 56.54
Количество сделок: 80 (разбивка по монетам: {'ADAUSDT': {'trades': 16, 'pnl': 5.50422192830471, 'percent_pnl': 18.347406427682365}, 'DOGEUSDT': {'trades': 15, 'pnl': -1.2412584086804594, 'percent_pnl': -4.137528028934865}, 'LINKUSDT': {'trades': 21, 'pnl': 2.0794031186057165, 'percent_pnl': 6.931343728685721}, 'SOLUSDT': {'trades': 15, 'pnl': 10.681883951219923, 'percent_pnl': 35.60627983739975}, 'TRBUSDT': {'trades': 13, 'pnl': 9.517557316126414, 'percent_pnl': 31.725191053754713}})
Средняя длительность сделок: 13.89 часов
Профитных сделок: 55 (по монетам: {'LINKUSDT': 15, 'ADAUSDT': 13, 'SOLUSDT': 11, 'TRBUSDT': 9, 'DOGEUSDT': 7})
Убыточных сделок: 25 (по монетам: {'DOGEUSDT': 8, 'LINKUSDT': 6, 'SOLUSDT': 4, 'TRBUSDT': 4, 'ADAUSDT': 3})

Месяц: 2020-11
Общий PnL: -18.01 (-60.04%)
Баланс на конец месяца: 38.53
Количество сделок: 92 (разбивка по монетам: {'ADAUSDT': {'trades': 12,

In [ ]:
# НАчало декабря

# Конфигурация теста
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '1h',
        'window_length': 100,
        'min_space': 5,
        'sloping_atr_length': 9,
        'take_profit_multiplier': 1.874,
        'stop_loss_multiplier': 2.097,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'BNBUSDT',
        'interval': '1h',
        'window_length': 34,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.00,
        'stop_loss_multiplier': 2.393,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'LINAUSDT',
        'interval': '1h',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 27,
        'take_profit_multiplier': 2.049,
        'stop_loss_multiplier': 2.058,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WIFUSDT',
        'interval': '1h',
        'window_length': 84,
        'min_space': 5,
        'sloping_atr_length': 7,
        'take_profit_multiplier': 2.003,
        'stop_loss_multiplier': 2.395,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WLDUSDT',
        'interval': '1h',
        'window_length': 68,
        'min_space': 5,
        'sloping_atr_length': 29,
        'take_profit_multiplier': 2.421,
        'stop_loss_multiplier': 2.551,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'XRPUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 17,
        'take_profit_multiplier': 1.758,
        'stop_loss_multiplier': 1.854,
        'trade_amount': 10,
        'leverage': 20
    },
]

start_date = '2023-10'
end_date = '2024-11'
initial_cash = 100

# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

Запуск тестирования для 1000PEPEUSDT...
Тестирование для 1000PEPEUSDT завершено.
Запуск тестирования для BNBUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для BNBUSDT завершено.
Запуск тестирования для LINAUSDT...
Тестирование для LINAUSDT завершено.
Запуск тестирования для ORDIUSDT...
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Тестирование для ORDIUSDT завершено.
Запуск тестирования для WIFUSDT...
Ошибка: Файл klines/WIFUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2023-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2023-12.zip поврежден или не является ZIP. Пропускаем.
Тестирование для WIFUSDT завершено.
Запуск тестирования для WLDUSDT...
Тестирование для WLDUSDT завершено.
Запуск тестирования для XRPUSDT...
Тестирование для XRPUSDT завершено.
Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.

===== Статистика по месяцам =====

Месяц: 2023-10
Общий PnL: 47.04 (47.04%)
Баланс на конец месяца: 147.04
Количество сделок: 65 (разбивка по монетам: {'1000PEPE

<ipython-input-41-941383555082>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))


Параметры моделей: 1000PEPEUSDT_30m (window=96, atr_length=10, min_space=5, take=2.0157635444989057, loss=2.56089497721195); BNBUSDT_1h (window=34, atr_length=20, min_space=5, take=1.0003717941136006, loss=2.39353768020516); LINAUSDT_1h (window=96, atr_length=27, min_space=5, take=2.049448110407128, loss=2.0580648628801086); ORDIUSDT_1h (window=42, atr_length=30, min_space=5, take=2.561307156240609, loss=1.0179889676917246); WIFUSDT_1h (window=84, atr_length=10, min_space=5, take=2.538796573657456, loss=2.591463669038581); WLDUSDT_1h (window=68, atr_length=29, min_space=5, take=2.42160013881293, loss=2.551619203185885); XRPUSDT_1h (window=42, atr_length=9, min_space=5, take=1.8071809921665287, loss=1.9214782453444468)


In [ ]:
# Конфигурация теста
# Начало декабря


coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.015,
        'stop_loss_multiplier': 2.560,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'BNBUSDT',
        'interval': '1h',
        'window_length': 34,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.00,
        'stop_loss_multiplier': 2.393,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'LINAUSDT',
        'interval': '1h',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 27,
        'take_profit_multiplier': 2.049,
        'stop_loss_multiplier': 2.058,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 10,
        'leverage': 20
    },
    # {
    #     'symbol': 'WIFUSDT',
    #     'interval': '1h',
    #     'window_length': 84,
    #     'min_space': 5,
    #     'sloping_atr_length': 10,
    #     'take_profit_multiplier': 2.538,
    #     'stop_loss_multiplier': 2.591,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
    # {
    #     'symbol': 'WLDUSDT',
    #     'interval': '1h',
    #     'window_length': 68,
    #     'min_space': 5,
    #     'sloping_atr_length': 29,
    #     'take_profit_multiplier': 2.421,
    #     'stop_loss_multiplier': 2.551,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
    {
        'symbol': 'XRPUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 9,
        'take_profit_multiplier': 1.807,
        'stop_loss_multiplier': 1.921,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'AAVEUSDT',
        'interval': '1h',
        'window_length': 40,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 2.068,
        'stop_loss_multiplier': 2.569,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'OPUSDT',
        'interval': '1h',
        'window_length': 80,
        'min_space': 5,
        'sloping_atr_length': 14,
        'take_profit_multiplier': 1.28,
        'stop_loss_multiplier': 1.408,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SOLUSDT',
        'interval': '1h',
        'window_length': 38,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 1.671,
        'stop_loss_multiplier': 1.239,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.973,
        'stop_loss_multiplier': 2.438,
        'trade_amount': 10,
        'leverage': 20
    },
    # {
    #     'symbol': 'TIAUSDT',
    #     'interval': '1h',
    #     'window_length': 54,
    #     'min_space': 5,
    #     'sloping_atr_length': 21,
    #     'take_profit_multiplier': 1.257,
    #     'stop_loss_multiplier': 2.332,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
#     {
#         'symbol': 'TIAUSDT',
#         'interval': '1h',
#         'window_length': 56,
#         'min_space': 5,
#         'sloping_atr_length': 20,
#         'take_profit_multiplier': 1.316,
#         'stop_loss_multiplier': 2.047,
#         'trade_amount': 10,
#         'leverage': 20
#     },

]

start_date = '2023-10'
end_date = '2024-11'
initial_cash = 100

# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

Запуск тестирования для 1000PEPEUSDT...
Тестирование для 1000PEPEUSDT завершено.
Запуск тестирования для BNBUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для BNBUSDT завершено.
Запуск тестирования для LINAUSDT...
Тестирование для LINAUSDT завершено.
Запуск тестирования для ORDIUSDT...
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Тестирование для ORDIUSDT завершено.
Запуск тестирования для XRPUSDT...
Тестирование для XRPUSDT завершено.
Запуск тестирования для AAVEUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для AAVEUSDT завершено.
Запуск тестирования для OPUSDT...
Тестирование для OPUSDT завершено.
Запуск тестирования для SOLUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Тестирование для SUIUSDT завершено.
Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.

===== Статистика по месяцам =====

Месяц: 2023-10
Общий PnL: -12.15 (-12.15%)
Баланс на конец месяца: 87.85
Количество сделок: 182 (разбивка по монетам: {'1000PEPEUSDT': {'trades': 27, 'pnl': -19.914922523508512, 'percent_pnl': -19.914922523508512}, 'AAVEUSDT': {'trades': 20, 'pnl': 1.7779318715654924, 'percent_pnl': 1.7779318715654924}, 'LINAUSDT': {'trades': 16, 'pnl': 1.096200644607555, 'percent_pnl': 1.096200644607555}, 'OPUSDT': {'trades': 25, 'pnl': -12.591938018177625, 'percent_pnl': -12.591938018177625}, 'SOLUSDT': {'trades': 34, 'pnl': -4.19361133233684, 'percent_pnl': -4.19361133233684}, 'SUIUSDT': {'trades': 38, 'pnl': 22.485519920776007, 'percent_pnl': 22.485519920776007}, 'XRPUSDT': {'trades': 22, 'pnl': -0.8127402904857002, 'percent_pnl': -0.8127402904857003}})
Средняя 

<ipython-input-41-941383555082>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))


In [ ]:
# Конфигурация теста
# Начало декабьряч
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.015,
        'stop_loss_multiplier': 2.560,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'BNBUSDT',
        'interval': '1h',
        'window_length': 34,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.00,
        'stop_loss_multiplier': 2.393,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'LINAUSDT',
        'interval': '1h',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 27,
        'take_profit_multiplier': 2.049,
        'stop_loss_multiplier': 2.058,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WIFUSDT',
        'interval': '1h',
        'window_length': 84,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.538,
        'stop_loss_multiplier': 2.591,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WLDUSDT',
        'interval': '1h',
        'window_length': 68,
        'min_space': 5,
        'sloping_atr_length': 29,
        'take_profit_multiplier': 2.421,
        'stop_loss_multiplier': 2.551,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'XRPUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 9,
        'take_profit_multiplier': 1.807,
        'stop_loss_multiplier': 1.921,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'AAVEUSDT',
        'interval': '1h',
        'window_length': 40,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 2.068,
        'stop_loss_multiplier': 2.569,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'OPUSDT',
        'interval': '1h',
        'window_length': 80,
        'min_space': 5,
        'sloping_atr_length': 14,
        'take_profit_multiplier': 1.28,
        'stop_loss_multiplier': 1.408,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SOLUSDT',
        'interval': '1h',
        'window_length': 38,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 1.671,
        'stop_loss_multiplier': 1.239,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.973,
        'stop_loss_multiplier': 2.438,
        'trade_amount': 10,
        'leverage': 20
    },
    # {
    #     'symbol': 'TIAUSDT',
    #     'interval': '1h',
    #     'window_length': 54,
    #     'min_space': 5,
    #     'sloping_atr_length': 21,
    #     'take_profit_multiplier': 1.257,
    #     'stop_loss_multiplier': 2.332,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
    {
        'symbol': 'TIAUSDT',
        'interval': '1h',
        'window_length': 56,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.316,
        'stop_loss_multiplier': 2.047,
        'trade_amount': 10,
        'leverage': 20
    },

]

start_date = '2023-10'
end_date = '2024-11'
initial_cash = 100

# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

Запуск тестирования для 1000PEPEUSDT...
Тестирование для 1000PEPEUSDT завершено.
Запуск тестирования для BNBUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для BNBUSDT завершено.
Запуск тестирования для LINAUSDT...
Тестирование для LINAUSDT завершено.
Запуск тестирования для ORDIUSDT...
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Тестирование для ORDIUSDT завершено.
Запуск тестирования для WIFUSDT...
Ошибка: Файл klines/WIFUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2023-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2023-12.zip поврежден или не является ZIP. Пропускаем.
Тестирование для WIFUSDT завершено.
Запуск тестирования для WLDUSDT...
Тестирование для WLDUSDT завершено.
Запуск тестирования для XRPUSDT...
Тестирование для XRPUSDT завершено.
Запуск тестирования для AAVEUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для AAVEUSDT завершено.
Запуск тестирования для OPUSDT...
Тестирование для OPUSDT завершено.
Запуск тестирования для SOLUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Тестирование для SUIUSDT завершено.
Запуск тестирования для TIAUSDT...
Тестирование для TIAUSDT завершено.
Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.

===== Статистика по месяцам =====

Месяц: 2023-10
Общий PnL: 15.79 (15.79%)
Баланс на конец месяца: 115.79
Количество сделок: 197 (разбивка по монетам: {'1000PEPEUSDT': {'trades': 27, 'pnl': -19.914922523508512, 'percent_pnl': -19.914922523508512}, 'AAVEUSDT': {'trades': 20, 'pnl': 1.7779318715654924, 'percent_pnl': 1.7779318715654924}, 'LINAUSDT': {'trades': 16, 'pnl': 1.096200644607555, 'percent_pnl': 1.096200644607555}, 'OPUSDT': {'trades': 25, 'pnl': -12.591938018177625, 'percent_pnl': -12.591938018177625}, 'SOLUSDT': {'trades': 34, 'pnl': -4.19361133233684, 'percent_pnl': -4.19361133233684}, 'SUIUSDT': {'trades': 38, 'pnl': 22.485519920776007, 'percent_pnl': 22.485519920776007}, 'WLDUSDT': {'trades': 15, 'pnl

<ipython-input-41-941383555082>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))


In [ ]:
# Конфигурация теста
# Начало декабьряч
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.015,
        'stop_loss_multiplier': 2.560,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'BNBUSDT',
        'interval': '1h',
        'window_length': 34,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.00,
        'stop_loss_multiplier': 2.393,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'LINAUSDT',
        'interval': '1h',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 27,
        'take_profit_multiplier': 2.049,
        'stop_loss_multiplier': 2.058,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WIFUSDT',
        'interval': '1h',
        'window_length': 84,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.538,
        'stop_loss_multiplier': 2.591,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WLDUSDT',
        'interval': '1h',
        'window_length': 68,
        'min_space': 5,
        'sloping_atr_length': 29,
        'take_profit_multiplier': 2.421,
        'stop_loss_multiplier': 2.551,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'XRPUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 9,
        'take_profit_multiplier': 1.807,
        'stop_loss_multiplier': 1.921,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'AAVEUSDT',
        'interval': '1h',
        'window_length': 40,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 2.068,
        'stop_loss_multiplier': 2.569,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'OPUSDT',
        'interval': '1h',
        'window_length': 80,
        'min_space': 5,
        'sloping_atr_length': 14,
        'take_profit_multiplier': 1.28,
        'stop_loss_multiplier': 1.408,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SOLUSDT',
        'interval': '1h',
        'window_length': 38,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 1.671,
        'stop_loss_multiplier': 1.239,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.973,
        'stop_loss_multiplier': 2.438,
        'trade_amount': 10,
        'leverage': 20
    },
    # {
    #     'symbol': 'TIAUSDT',
    #     'interval': '1h',
    #     'window_length': 54,
    #     'min_space': 5,
    #     'sloping_atr_length': 21,
    #     'take_profit_multiplier': 1.257,
    #     'stop_loss_multiplier': 2.332,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
    {
        'symbol': 'TIAUSDT',
        'interval': '1h',
        'window_length': 56,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.316,
        'stop_loss_multiplier': 2.047,
        'trade_amount': 10,
        'leverage': 20
    },

]

start_date = '2023-10'
end_date = '2024-11'
initial_cash = 100

# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)